In [2]:
import pandas as pd
import numpy as np
import random
from typing import Dict, List, Any, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# === 1. CHARGEMENT ET NETTOYAGE DES DONNÉES ===

def load_and_clean_data(file_path: str) -> pd.DataFrame:
    """Charge et nettoie les données TAPE avec gestion robuste des erreurs"""
    try:
        agent_data = pd.read_excel(file_path)
        
        # Colonnes à supprimer avec vérification
        columns_to_drop = ['general', 'netrev_pcapita', 'value_added_pcapita', 'inc_person_day']
        existing_columns_to_drop = [col for col in columns_to_drop if col in agent_data.columns]
        agent_data = agent_data.drop(columns=existing_columns_to_drop)
        
        # Gestion des valeurs manquantes
        if 'Village' in agent_data.columns:
            agent_data['Village'] = agent_data['Village'].fillna(agent_data['Village'].mode()[0] if not agent_data['Village'].mode().empty else 'Unknown')
        
        # Imputation des valeurs quantitatives
        quantitative_cols_with_nan = agent_data.select_dtypes(include=[np.number]).columns[agent_data.select_dtypes(include=[np.number]).isnull().any()].tolist()
        for col in quantitative_cols_with_nan:
            if col != 'Village':
                agent_data[col] = agent_data[col].fillna(agent_data[col].mean())
        
        return agent_data
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement des données: {e}")
        return create_complete_agent_data(285)

# === 2. CLASSE EXPLOITATIONTAPE ===

class ExploitationTAPE:
    """Classe unifiée pour représenter une exploitation agricole avec toutes les descriptions réelles des variables TAPE."""
    
    def __init__(self, unique_id: int, model, data_row: pd.Series):
        self.unique_id = unique_id
        self.model = model
        self.zone_type = getattr(model, 'zone_name', 'OPIB')
        
        self._initialize_from_real_data(data_row)
        self._initialize_behavioral_attributes()
        self._initialize_agent_state()
        
        self.initial_caet_tot = getattr(self, 'caet_tot', 40.0)
        self.initial_technologies = self.ae_technologies.copy()
        
    def _initialize_from_real_data(self, data_row: pd.Series):
        """Initialise tous les indicateurs à partir des données réelles de l'agent."""
        self.original_data = data_row.to_dict()
        self.ae_technologies = {}
        
        # Dictionnaire complet des technologies agroécologiques
        all_ae_indicators = {
            # Diversité
            "crops": "CULTURES - Échelle 0-4", "animals": "ANIMAUX - Échelle 0-4", 
            "trees": "ARBRES - Échelle 0-4", "animnum": "DIVERSITÉ DES ACTIVITÉS, PRODUITS ET SERVICES - Échelle 0-4",
            # Synergies
            "cla_int": "INTÉGRATION CULTURES-ÉLEVAGE-AQUACULTURE - Échelle 0-4", "s_plant": "GESTION DU SYSTÈME SOL-PLANTES - Échelle 0-4",
            "tree_int": "INTEGRATION AVEC LES ARBRES - Échelle 0-4", "connectivity": "CONNECTIVITÉ ENTRE LES ÉLÉMENTS - Échelle 0-4",
            # Efficience
            "ext_inp": "UTILISATION D'INTRANTS EXTERIEURS - Échelle 0-4", "soil_fert": "GESTION DE LA FERTILITÉ DU SOL - Échelle 0-4",
            "pest_dis": "GESTION DES PESTES ET DES MALADIES - Échelle 0-4", "vuln": "PRODUCTIVITÉ ET BESOINS DU MÉNAGE - Échelle 0-4",
            # Recyclage
            "rec_biomass": "RECYCLAGE DE LA BIOMASSE ET DES NUTRIENTS - Échelle 0-4", "water": "PRÉSERVATION ET CONSERVATION DE L'EAU - Échelle 0-4",
            "seeds_breeds": "GESTION DES GRAINES ET DES RACES ANIMALES - Échelle 0-4", "ren_energy": "ENERGIE RENOUVELABLE - Échelle 0-4",
            # Résilience
            "waste": "STABILITÉ DE LA PRODUCTION ET CAPACITÉ À RÉSISTER AUX PERTURBATIONS - Échelle 0-4", "indebt": "ENDETTEMENT - Échelle 0-4",
            # Culture & Traditions
            "diet": "RÉGIME ALIMENTAIRE APPROPRIÉ ET CONSCIENCE NUTRITIONNELLE - Échelle 0-4", "food-heritage": "IDENTITÉ ET CONSCIENCE LOCALES OU TRADITIONNELLES - Échelle 0-4",
            "food-self-suff": "UTILISATION DE VARIETÉS/RACES LOCALES - Échelle 0-4",
            # Co-création
            "platforms": "PLATEFORMES POUR LA CRÉATION ET LE TRANSFERT DE CONNAISSANCES - Échelle 0-4", "ae_know": "ACCÈS AUX CONNAISSANCES AGROÉCOLOGIQUES - Échelle 0-4",
            "partic_orgs": "PARTICIPATION AUX RÉSEAUX ET ORGANISATIONS - Échelle 0-4",
            # Valeurs sociales
            "women": "ÉMANCIPATION DES FEMMES - Échelle 0-4", "labour": "TRAVAIL (CONDITIONS DE PRODUCTION) - Échelle 0-4",
            "youth": "ÉMANCIPATION DE LA JEUNESSE - Échelle 0-4", "animalwel": "BIEN-ÊTRE ANIMAL - Échelle 0-4",
            # Économie circulaire
            "mkt_local": "PRODUITS COMMERCIALISÉS SUR LE MARCHÉ LOCAL - Échelle 0-4", "networks": "RÉSEAUX DE PRODUCTEURS - Échelle 0-4",
            "local_fs": "SYSTÈME ALIMENTAIRE LOCAL - Échelle 0-4",
            # Gouvernance
            "prod_empow": "ÉMANCIPATION DES PRODUCTEURS - Échelle 0-4", "prod_orgs": "ORGANISATIONS DE PRODUCTEURS - Échelle 0-4",
            "partic_prod": "PARTICIPATION DANS LA GOUVERNANCE - Échelle 0-4",
            # Autres
            "coalanwel": "BIEN-ÊTRE GLOBAL - Échelle 0-4"
        }
        
        for tech, description in all_ae_indicators.items():
            if tech in data_row and not pd.isna(data_row[tech]):
                self.ae_technologies[tech] = float(data_row[tech])
            else:
                self.ae_technologies[tech] = self._get_realistic_tech_value(tech)
        
        # Composition familiale
        self.family_composition = {}
        family_indicators = ["hh_men", "hh_women", "hh_myoung", "hh_fyoung", "hh_children", "hh_fem"]
        for member in family_indicators:
            if member in data_row and not pd.isna(data_row[member]):
                self.family_composition[member] = float(data_row[member])
            else:
                self.family_composition[member] = self._get_realistic_family_value(member)
        
        # Taille de l'exploitation
        if "area" in data_row and not pd.isna(data_row["area"]):
            self.farm_area = float(data_row["area"])
        else:
            self.farm_area = random.uniform(1.0, 50.0)
        
        # Indicateurs CAET
        caet_indicators = ['caet_tot', 'div_score', 'syn_score', 'eff_score', 'rec_score', 'res_score', 'cocr_score', 'human_score', 'circ_score', 'respg_score', 'cultf_score']
        for indicator in caet_indicators:
            if indicator in data_row and not pd.isna(data_row[indicator]):
                setattr(self, indicator, float(data_row[indicator]))
            else:
                setattr(self, indicator, self._get_realistic_default(indicator))
        
        # Indicateurs de performance
        performance_indicators = [
            'farm_revenue', 'value_added', 'tot_productivity', 'market_access', 'input_costs', 'profit_margin', 
            'soil_health', 'pest_score', 'dietary_diversity', 'Agricultural_diversity_tape', 'water_management',
            'carbon_sequestration', 'biodiversity_index', 'ecological_resilience', 'youth_score', 'landtenure', 
            'wemp_score', 'social_cohesion', 'knowledge_sharing', 'community_participation', 'local_market_integration',
            'value_chain_development', 'infrastructure_access', 'territorial_connectivity', 'circular_economy', 'food_sovereignty'
        ]
        
        for indicator in performance_indicators:
            if indicator in data_row and not pd.isna(data_row[indicator]):
                setattr(self, indicator, float(data_row[indicator]))
            else:
                setattr(self, indicator, self._get_realistic_default(indicator))

    def _get_realistic_tech_value(self, tech: str) -> float:
        """Retourne une valeur réaliste pour une technologie agroécologique (0-4)."""
        tech_difficulty = {
            "crops": [0.1, 0.2, 0.3, 0.25, 0.15], "animals": [0.15, 0.25, 0.3, 0.2, 0.1],
            "animnum": [0.1, 0.25, 0.35, 0.2, 0.1], "diet": [0.1, 0.2, 0.3, 0.25, 0.15],
            "s_plant": [0.15, 0.25, 0.3, 0.2, 0.1], "soil_fert": [0.2, 0.3, 0.25, 0.15, 0.1],
            "pest_dis": [0.15, 0.35, 0.3, 0.15, 0.05], "water": [0.25, 0.3, 0.25, 0.15, 0.05],
            "mkt_local": [0.2, 0.3, 0.25, 0.15, 0.1], "cla_int": [0.2, 0.3, 0.25, 0.15, 0.1],
            "tree_int": [0.25, 0.3, 0.25, 0.15, 0.05], "connectivity": [0.3, 0.3, 0.25, 0.1, 0.05],
            "ext_inp": [0.1, 0.4, 0.3, 0.15, 0.05], "ren_energy": [0.4, 0.3, 0.2, 0.08, 0.02],
            "seeds_breeds": [0.3, 0.35, 0.2, 0.1, 0.05], "platforms": [0.25, 0.3, 0.25, 0.15, 0.05],
            "ae_know": [0.2, 0.3, 0.25, 0.15, 0.1], "women": [0.2, 0.3, 0.25, 0.15, 0.1],
            "prod_empow": [0.25, 0.3, 0.25, 0.15, 0.05], "rec_biomass": [0.2, 0.3, 0.25, 0.15, 0.1],
            "waste": [0.25, 0.3, 0.25, 0.15, 0.05], "indebt": [0.1, 0.2, 0.3, 0.25, 0.15],
            "youth": [0.2, 0.3, 0.25, 0.15, 0.1], "networks": [0.25, 0.3, 0.25, 0.15, 0.05],
            "local_fs": [0.3, 0.3, 0.25, 0.1, 0.05], "prod_orgs": [0.25, 0.3, 0.25, 0.15, 0.05],
            "partic_prod": [0.3, 0.3, 0.25, 0.1, 0.05]
        }
        
        weights = tech_difficulty.get(tech, [0.2, 0.3, 0.25, 0.15, 0.1])
        return float(random.choices([0, 1, 2, 3, 4], weights=weights)[0])

    def _get_realistic_family_value(self, member: str) -> float:
        """Retourne une valeur réaliste pour la composition familiale."""
        if member == "hh_men": return float(random.randint(1, 3))
        elif member == "hh_women": return float(random.randint(1, 2))
        elif member in ["hh_myoung", "hh_fyoung"]: return float(random.randint(0, 2))
        elif member == "hh_children": return float(random.randint(0, 4))
        elif member == "hh_fem": return float(random.randint(1, 3))
        else: return 0.0

    def _get_realistic_default(self, indicator: str) -> float:
        """Retourne une valeur par défaut réaliste pour les autres indicateurs."""
        base_values = {
            'caet_tot': 40.0, 'div_score': 45.0, 'syn_score': 43.0, 'eff_score': 47.0, 'rec_score': 38.0,
            'res_score': 44.0, 'cocr_score': 46.0, 'human_score': 49.0, 'circ_score': 42.0, 'respg_score': 45.0,
            'cultf_score': 40.0, 'farm_revenue': 1700000, 'value_added': 30.0, 'profit_margin': 25.0,
            'input_costs': 35.0, 'market_access': 55.0, 'soil_health': 57.0, 'pest_score': 40.0,
            'biodiversity_index': 30.0, 'carbon_sequestration': 35.0, 'water_management': 50.0,
            'ecological_resilience': 40.0, 'social_cohesion': 58.0, 'knowledge_sharing': 48.0,
            'community_participation': 50.0, 'youth_score': 37.0, 'landtenure': 70.0, 'wemp_score': 40.0,
            'local_market_integration': 42.0, 'value_chain_development': 38.0, 'infrastructure_access': 47.0,
            'territorial_connectivity': 45.0, 'circular_economy': 32.0, 'food_sovereignty': 35.0,
            'dietary_diversity': 50.0, 'Agricultural_diversity_tape': 42.0, 'tot_productivity': 52.0
        }
        
        base_value = base_values.get(indicator, 40.0)
        multiplier = random.uniform(0.9, 1.3) if self.zone_type == "OHVN" else random.uniform(0.7, 1.1)
        return base_value * multiplier

    def _calculate_agroecology_score(self) -> float:
        """Calcule un score agroécologique global basé sur les technologies (0-100)."""
        if not self.ae_technologies: return 30.0
        total_score = sum(self.ae_technologies.values())
        max_possible = len(self.ae_technologies) * 4.0
        return (total_score / max_possible) * 100 if max_possible > 0 else 30.0

    def _calculate_technology_adoption_level(self) -> float:
        """Calcule le niveau d'adoption des technologies agroécologiques (0-100%)."""
        if not self.ae_technologies: return 0.0
        adopted_techs = sum(1 for score in self.ae_technologies.values() if score >= 2.0)
        return (adopted_techs / len(self.ae_technologies)) * 100

    def _calculate_family_labor_score(self) -> float:
        """Calcule un score de main-d'œuvre familiale (0-100)."""
        labor_weights = {"hh_men": 1.0, "hh_women": 0.9, "hh_fem": 0.9, "hh_myoung": 0.8, "hh_fyoung": 0.8, "hh_children": 0.3}
        weighted_labor = sum(self.family_composition.get(member, 0) * weight for member, weight in labor_weights.items())
        labor_per_hectare = weighted_labor / max(1.0, self.farm_area)
        return min(100.0, labor_per_hectare * 20.0)

    def _initialize_behavioral_attributes(self):
        """Initialise les attributs comportementaux basés sur les données réelles."""
        self.momentum = {}
        self.stagnation_counter = {}
        self.crisis_impact = 1.0
        
        ae_score = self._calculate_agroecology_score()
        base_innovation = 0.3 + (ae_score / 100) * 0.4
        base_social = 0.2 + (ae_score / 100) * 0.5
        base_community = 0.3 + (ae_score / 100) * 0.4
        base_thinking = 0.3 + (ae_score / 100) * 0.4
        
        self.innovation_capacity = np.clip(random.uniform(base_innovation - 0.1, base_innovation + 0.1), 0.2, 1.0)
        self.social_network = np.clip(random.uniform(base_social - 0.1, base_social + 0.1), 0.1, 0.9)
        self.community_engagement = np.clip(random.uniform(base_community - 0.1, base_community + 0.1), 0.2, 0.9)
        self.systemic_thinking = np.clip(random.uniform(base_thinking - 0.1, base_thinking + 0.1), 0.2, 0.9)
        self.adoption_stability = random.uniform(0.6, 0.9)
        self.risk_aversion = random.uniform(0.3, 0.8)
        
        if self.zone_type == "OHVN":
            self.innovation_capacity = np.clip(self.innovation_capacity + 0.1, 0.2, 1.0)
            self.social_network = np.clip(self.social_network + 0.1, 0.1, 0.9)
        else:
            self.community_engagement = np.clip(self.community_engagement + 0.1, 0.2, 0.9)
            self.systemic_thinking = np.clip(self.systemic_thinking + 0.1, 0.2, 0.9)

    def _initialize_agent_state(self):
        """Initialise l'état de l'agent basé sur les technologies, famille et taille."""
        ae_score = self._calculate_agroecology_score()
        tech_adoption_level = self._calculate_technology_adoption_level()
        family_labor_score = self._calculate_family_labor_score()
        
        base_prob = 0.2
        tech_factor = (tech_adoption_level / 100) * 0.4
        labor_factor = (family_labor_score / 100) * 0.3
        farm_size_factor = self._calculate_farm_size_factor() * 0.2
        zone_factor = 0.1 if self.zone_type == "OHVN" else 0.0
        caet_factor = (getattr(self, 'caet_tot', 40) / 100) * 0.1
        
        self.adoption_probability = max(0.05, min(0.95, base_prob + tech_factor + labor_factor + farm_size_factor + zone_factor + caet_factor))
        
        existing_adoption = getattr(self, 'adopts_agroecology', None)
        if existing_adoption is not None:
            self.adopts_agroecology = bool(existing_adoption)
        else:
            adoption_threshold = 0.6
            self.adopts_agroecology = (tech_adoption_level > 50 or ae_score > 60 or random.random() < self.adoption_probability)
        
        all_indicators = [attr for attr in dir(self) if not attr.startswith('_') and not callable(getattr(self, attr)) and isinstance(getattr(self, attr), (int, float))]
        for indicator in all_indicators:
            self.momentum[indicator] = random.uniform(0.8, 1.2)
            self.stagnation_counter[indicator] = 0

    def _calculate_farm_size_factor(self) -> float:
        """Calcule l'influence de la taille de l'exploitation sur l'adoption."""
        if self.farm_area < 5: return 0.3
        elif self.farm_area < 20: return 0.1
        else: return -0.1

    def step(self, scenario_effects: Dict[str, Any]):
        """Méthode de simulation pour un pas de temps avec évolution réaliste."""
        policy_effect = self._calculate_policy_effect(scenario_effects)
        climate_impact = scenario_effects.get("climate", 1.0) * random.uniform(0.9, 1.1)
        adoption_boost = scenario_effects.get("adoption_boost", 0.0)
        
        self._update_agroecology_adoption(adoption_boost, policy_effect)
        self._update_technologies_with_evolution(policy_effect, climate_impact)
        self._update_caet_scores_with_evolution(policy_effect, climate_impact, scenario_effects)
        self._update_performance_indicators_with_evolution(policy_effect, climate_impact, scenario_effects)
        self._apply_synergy_effects()
        self._apply_realistic_constraints()

    def _calculate_policy_effect(self, scenario_effects: Dict[str, Any]) -> float:
        """Calcule l'effet des politiques."""
        policy_map = {"highly_supportive": 1.3, "supportive": 1.15, "neutral": 1.0, "restrictive": 0.8}
        base_effect = policy_map.get(scenario_effects.get("policy", "neutral"), 1.0)
        zone_multiplier = 1.05 if self.zone_type == "OHVN" else 0.95
        return base_effect * zone_multiplier

    def _update_agroecology_adoption(self, adoption_boost: float, policy_effect: float):
        """Met à jour l'adoption avec une logique plus réaliste et progressive."""
        if not self.adopts_agroecology:
            tech_level = self._calculate_technology_adoption_level()
            economic_score = self._calculate_economic_situation()
            family_labor = self._calculate_family_labor_score()
            
            base_prob = 0.005
            tech_factor = min(0.2, (tech_level / 100) * 0.4)
            economic_factor = min(0.15, (economic_score / 100) * 0.3)
            labor_factor = min(0.1, (family_labor / 100) * 0.2)
            social_factor = min(0.1, self.social_network * 0.2)
            policy_factor = min(0.15, (policy_effect - 1.0) * 0.3)
            boost_factor = min(0.1, adoption_boost * 0.2)
            
            risk_factor = self.risk_aversion * 0.2
            farm_size_penalty = 0.1 if self.farm_area > 20 else 0.2 if self.farm_area > 50 else 0.0
            
            adoption_prob = (base_prob + tech_factor + economic_factor + labor_factor + social_factor + policy_factor + boost_factor)
            adoption_prob *= (1 - risk_factor - farm_size_penalty)
            adoption_prob = min(0.15, max(0.001, adoption_prob))
            
            if random.random() < adoption_prob:
                self.adopts_agroecology = True
                self._on_agroecology_adoption()
        else:
            abandonment_prob = 0.002 * (1 - self.adoption_stability)
            economic_pressure = 0.1 if hasattr(self, 'profit_margin') and self.profit_margin < 10 else 0.2 if hasattr(self, 'profit_margin') and self.profit_margin < 5 else 0.0
            climate_pressure = 0.1 if hasattr(self, 'crisis_impact') and self.crisis_impact > 1.2 else 0.0
            risk_factor = self.risk_aversion * 0.15
            
            abandonment_prob *= (1 + economic_pressure + climate_pressure + risk_factor)
            abandonment_prob = min(0.05, abandonment_prob)
            
            if random.random() < abandonment_prob:
                self.adopts_agroecology = False
                self.innovation_capacity = max(0.1, self.innovation_capacity - 0.05)
                self.social_network = max(0.1, self.social_network - 0.03)

    def _calculate_economic_situation(self) -> float:
        """Calcule la situation économique de l'agent de manière plus réaliste."""
        economic_indicators = []
        
        if hasattr(self, 'farm_revenue'):
            if self.farm_revenue > 5000000: revenue_score = 100
            elif self.farm_revenue > 2000000: revenue_score = 80
            elif self.farm_revenue > 1000000: revenue_score = 60
            elif self.farm_revenue > 500000: revenue_score = 40
            else: revenue_score = 20
            economic_indicators.append(revenue_score * 0.4)
        
        if hasattr(self, 'profit_margin'):
            if self.profit_margin > 30: margin_score = 100
            elif self.profit_margin > 20: margin_score = 80
            elif self.profit_margin > 10: margin_score = 60
            elif self.profit_margin > 0: margin_score = 40
            else: margin_score = 20
            economic_indicators.append(margin_score * 0.3)
        
        if hasattr(self, 'value_added'):
            economic_indicators.append(min(100, self.value_added) * 0.3)
        
        return np.mean(economic_indicators) if economic_indicators else 30.0

    def _update_technologies_with_evolution(self, policy_effect: float, climate_impact: float):
        """Met à jour les technologies avec une évolution plus réaliste et limitée."""
        if not self.adopts_agroecology:
            for tech_name in self.ae_technologies:
                if random.random() < 0.05:
                    degradation = random.uniform(0.01, 0.03)
                    self.ae_technologies[tech_name] = max(0, self.ae_technologies[tech_name] - degradation)
            return
        
        for tech_name in self.ae_technologies:
            current_level = self.ae_technologies[tech_name]
            if current_level < 4.0:
                base_improvement = random.uniform(0.005, 0.03)
                innovation_factor = 1.0 + (self.innovation_capacity * 0.3)
                policy_factor = min(1.5, policy_effect)
                climate_factor = climate_impact
                social_factor = 1.0 + (self.social_network * 0.2)
                
                improvement = base_improvement * innovation_factor * policy_factor * climate_factor * social_factor
                distance_to_max = ((4.0 - current_level) / 4.0) ** 2
                improvement *= distance_to_max
                
                if current_level > 3.0: improvement *= 0.5
                if current_level > 3.5: improvement *= 0.3
                
                new_level = current_level + improvement
                self.ae_technologies[tech_name] = min(3.9, new_level)

    def _update_caet_scores_with_evolution(self, policy_effect: float, climate_impact: float, scenario_effects: Dict[str, Any]):
        """Met à jour les scores CAET avec une évolution plus réaliste et limitée."""
        adoption_factor = 1.2 if self.adopts_agroecology else 0.9
        tech_level = self._calculate_technology_adoption_level()
        tech_factor = 1.0 + (tech_level / 100) * 0.3
        
        scenario_growth = {
            'actuel': 0.005, 'transition': 0.01, 'baguineda_to_be': 0.015, 'tiele_to_be': 0.015,
            'baguineda_gnogon_te': 0.02, 'ohvn_ecosysteme_resilient': 0.018, 'baguineda_baa_kedi': 0.002,
            'ohvn_territoires_fragmentes': 0.003
        }
        
        base_growth_rate = scenario_growth.get(getattr(self.model, 'scenario_type', 'actuel'), 0.005)
        
        caet_weights = {
            'div_score': 0.14, 'syn_score': 0.12, 'eff_score': 0.09, 'rec_score': 0.08, 'res_score': 0.16,
            'cultf_score': 0.06, 'cocr_score': 0.09, 'human_score': 0.11, 'circ_score': 0.08, 'respg_score': 0.07
        }
        
        for dimension, weight in caet_weights.items():
            if hasattr(self, dimension):
                current_value = getattr(self, dimension)
                dimension_growth = base_growth_rate * weight * 8
                growth_rate = dimension_growth * adoption_factor * tech_factor * min(1.3, policy_effect) * climate_impact
                random_variation = random.uniform(-0.002, 0.002)
                
                ceiling_effect = 1.0
                if current_value > 80: ceiling_effect = 0.7
                if current_value > 90: ceiling_effect = 0.4
                if current_value > 95: ceiling_effect = 0.1
                
                growth_rate *= ceiling_effect
                new_value = current_value * (1 + growth_rate + random_variation)
                
                if dimension in ['div_score', 'syn_score'] and self.farm_area < 2:
                    new_value = min(new_value, 75)
                elif dimension == 'rec_score' and self.farm_area > 50:
                    new_value = min(new_value, 70)
                
                new_value = min(98.0, new_value)
                setattr(self, dimension, max(0, new_value))
        
        self._update_total_caet()

    def _update_total_caet(self):
        """Calcule le score CAET total avec des limites réalistes."""
        weights = {
            'div_score': 0.14, 'syn_score': 0.12, 'eff_score': 0.09, 'rec_score': 0.08, 'res_score': 0.16,
            'cultf_score': 0.06, 'cocr_score': 0.09, 'human_score': 0.11, 'circ_score': 0.08, 'respg_score': 0.07
        }
        
        scores = []
        total_weight = 0
        
        for indicator, weight in weights.items():
            if hasattr(self, indicator):
                scores.append(getattr(self, indicator) * weight)
                total_weight += weight
        
        if total_weight > 0:
            calculated_caet = sum(scores) / total_weight
            
            tech_level = self._calculate_technology_adoption_level()
            if tech_level > 50: calculated_caet *= 1.01
            if tech_level > 70: calculated_caet *= 1.02
            
            if calculated_caet > 85: calculated_caet = 85 + (calculated_caet - 85) * 0.5
            if calculated_caet > 90: calculated_caet = 90 + (calculated_caet - 90) * 0.3
            if calculated_caet > 95: calculated_caet = 95 + (calculated_caet - 95) * 0.1
            
            self.caet_tot = min(97.0, calculated_caet)

    def _update_performance_indicators_with_evolution(self, policy_effect: float, climate_impact: float, scenario_effects: Dict[str, Any]):
        """Met à jour les indicateurs de performance avec évolution."""
        agroeco_factor = 1.2 if self.adopts_agroecology else 0.9
        
        if hasattr(self, 'tot_productivity'):
            base_growth = random.uniform(0.005, 0.015)
            growth = base_growth * policy_effect * agroeco_factor * (1.0 + self.innovation_capacity * 0.2)
            self.tot_productivity = min(100, self.tot_productivity * (1 + growth))
        
        if hasattr(self, 'farm_revenue'):
            base_growth = random.uniform(0.008, 0.025)
            market_factor = 1.0 + (getattr(self, 'market_access', 0.5) * 0.4)
            growth = base_growth * policy_effect * market_factor * agroeco_factor
            self.farm_revenue = max(0, self.farm_revenue * (1 + growth))
        
        if hasattr(self, 'soil_health'):
            base_growth = random.uniform(0.003, 0.012)
            growth = base_growth * policy_effect * climate_impact * agroeco_factor
            self.soil_health = min(100, self.soil_health * (1 + growth))
        
        if hasattr(self, 'biodiversity_index'):
            base_growth = random.uniform(0.002, 0.010)
            growth = base_growth * policy_effect * agroeco_factor
            self.biodiversity_index = min(100, self.biodiversity_index * (1 + growth))
        
        if hasattr(self, 'ecological_resilience'):
            base_growth = random.uniform(0.004, 0.015)
            diversity_bonus = len([tech for tech, level in self.ae_technologies.items() if level >= 2]) / len(self.ae_technologies) * 0.3
            growth = base_growth * policy_effect * (1 + diversity_bonus) * agroeco_factor
            self.ecological_resilience = min(100, self.ecological_resilience * (1 + growth))

    def _on_agroecology_adoption(self):
        """Effets lors de l'adoption de l'agroécologie."""
        for tech_name in self.ae_technologies:
            if random.random() < 0.4:
                improvement = random.uniform(0.2, 0.8)
                self.ae_technologies[tech_name] = min(4.0, self.ae_technologies[tech_name] + improvement)
        
        self.innovation_capacity = min(1.0, self.innovation_capacity + 0.1)
        self.social_network = min(0.9, self.social_network + 0.05)

    def _apply_synergy_effects(self):
        """Applique les effets de synergie."""
        synergies = {
            ('div_score', 'syn_score'): 0.015, ('eff_score', 'rec_score'): 0.010,
            ('res_score', 'circ_score'): 0.018, ('cocr_score', 'respg_score'): 0.012,
            ('cultf_score', 'human_score'): 0.008
        }
        
        for (dim1, dim2), synergy_factor in synergies.items():
            if (hasattr(self, dim1) and hasattr(self, dim2) and getattr(self, dim1) > 45 and getattr(self, dim2) > 45):
                bonus1 = getattr(self, dim2) * synergy_factor * 0.01
                bonus2 = getattr(self, dim1) * synergy_factor * 0.01
                setattr(self, dim1, min(getattr(self, dim1) + bonus1, 100))
                setattr(self, dim2, min(getattr(self, dim2) + bonus2, 100))

    def _apply_realistic_constraints(self):
        """Applique des contraintes réalistes."""
        if hasattr(self, 'caet_tot') and self.caet_tot > 85:
            slowdown_factor = max(0.3, 1.0 - (self.caet_tot - 85) / 15)
            for indicator in ['div_score', 'syn_score', 'eff_score', 'res_score']:
                if hasattr(self, indicator):
                    current = getattr(self, indicator)
                    setattr(self, indicator, current * slowdown_factor + current * (1 - slowdown_factor) * 0.95)

    def get_agent_data(self) -> Dict[str, Any]:
        """Retourne toutes les données de l'agent."""
        data = {
            'unique_id': self.unique_id, 'zone': self.zone_type, 'adopts_agroecology': self.adopts_agroecology,
            'adoption_probability': self.adoption_probability, 'innovation_capacity': self.innovation_capacity,
            'social_network': self.social_network, 'community_engagement': self.community_engagement,
            'systemic_thinking': self.systemic_thinking, 'risk_aversion': self.risk_aversion,
            'crisis_impact': self.crisis_impact, 'farm_area': self.farm_area,
            'ae_technology_score': self._calculate_agroecology_score(),
            'technology_adoption_level': self._calculate_technology_adoption_level(),
            'family_labor_score': self._calculate_family_labor_score(),
            'momentum_avg': np.mean(list(self.momentum.values())) if self.momentum else 1.0,
            'stagnation_avg': np.mean(list(self.stagnation_counter.values())) if self.stagnation_counter else 0,
        }
        
        for member, count in self.family_composition.items():
            data[member] = count
        
        for tech, level in self.ae_technologies.items():
            data[f"tech_{tech}"] = level
        
        all_attrs = [attr for attr in dir(self) if not attr.startswith('_') and not callable(getattr(self, attr)) and isinstance(getattr(self, attr), (int, float, bool, str)) and attr not in data]
        for attr in all_attrs:
            data[attr] = getattr(self, attr)
        
        return data

    def __repr__(self) -> str:
        tech_level = self._calculate_technology_adoption_level()
        return (f"ExploitationTAPE(id={self.unique_id}, zone={self.zone_type}, "
                f"CAET={getattr(self, 'caet_tot', 'N/A'):.1f}, TechLevel={tech_level:.1f}%, "
                f"Agroécologie={self.adopts_agroecology}, Superficie={self.farm_area:.1f}ha)")

# === 3. CLASSES POUR LE SYSTÈME PROSPECTIF ===

from enum import Enum
from dataclasses import dataclass

class ZoneType(Enum):
    OPIB = "OPIB"
    OHVN = "OHVN"

class TypeScenario(Enum):
    TENDANCIEL = "tendanciel"
    DESIRABLE = "desirable"
    ALTERNATIF = "alternatif"
    CRITIQUE = "critique"

@dataclass
class ForceMotrice:
    code: str
    nom: str
    definition: str
    etats_futurs: Dict[str, str]

@dataclass
class ScenarioProspectif:
    nom: str
    code: str
    zone: ZoneType
    type_scenario: TypeScenario
    description: str
    configuration: Dict[str, str]
    agroecologie_place: str
    leviers: List[str]
    contraintes: List[str]
    points_basculement: List[str]

# === 4. MODÈLE ZONEAGROECOLOGICALMODEL ===

class ZoneAgroecologicalModel:
    """Modèle de simulation agroécologique unifié avec prospective territoriale."""
    
    PERFORMANCE_INDICATORS = {
        "dietary_diversity": "Dietary Diversity", "soil_health": "Soil Health", "landtenure": "Land Tenure", 
        "pest_score": "Pesticides", "Agricultural_diversity_tape": "Biodiversity", "youth_score": "Youth",
        "tot_productivity": "Productivity", "farm_revenue": "Income", "value_added": "Value Added", 
        "wemp_score": "Women", "caet_tot": "Performance Globale"
    }
    
    SCENARIO_MULTIPLIERS = {
        "actuel": 0.8, "transition": 1.2, "baguineda_to_be": 1.5, "tiele_to_be": 1.3
    }
    
    def __init__(self, zone_name: str, agent_data: pd.DataFrame, scenario_type: str = "actuel", num_agents: int = 50, start_year: int = 2025):
        self.zone_name = zone_name
        self.scenario_type = scenario_type
        self.num_agents = num_agents
        self.start_year = start_year
        self.current_year = start_year
        self.current_step = 0
        
        self._initialize_prospective_system()
        self._prepare_agent_data(agent_data)
        self._initialize_agents()
        self._analyze_agent_situations()
        self._initialize_tracking_systems()
        
        print(f"✅ Modèle {zone_name} initialisé - {len(self.all_agents)} agents - Scénario: {scenario_type}")

    def _initialize_prospective_system(self):
        """Initialise le système prospectif de manière robuste."""
        try:
            self.forces_motrices = self._define_forces_motrices()
            self.scenarios_prospectifs = self._define_scenarios_prospectifs()
            self.scenario_prospectif = self._get_scenario_prospectif(self.scenario_type)
            self.scenario_effects = self._define_scenario_effects_from_prospective()
            
            self.scenarios_par_zone = {ZoneType.OPIB: [], ZoneType.OHVN: []}
            
            if self.zone_name == "OPIB":
                self.charger_scenarios_opib()
            else:
                self.generer_scenarios_ohvn_par_miroir()
                
        except Exception as e:
            print(f"⚠️ Erreur lors de l'initialisation prospective: {e}")
            self._initialize_fallback_system()

    def _initialize_fallback_system(self):
        """Système de secours en cas d'erreur."""
        self.scenario_effects = {
            "policy": "neutral", "climate": 1.0, "adoption_boost": 0.0, "regression_effects": {},
            "territorial_effects": {}, "description": "Scénario par défaut"
        }

    def charger_scenarios_opib(self):
        """Charge les scénarios OPIB du document de référence"""
        try:
            scenarios_opib = [
                ScenarioProspectif(
                    nom="Baguineda gnogon tè - Baguineda est le meilleur", code="A1B1C1D1E1F1G1", zone=ZoneType.OPIB, type_scenario=TypeScenario.DESIRABLE,
                    description="Scénario idéal avec toutes les forces motrices dans leur état optimal",
                    configuration={"A": "A1", "B": "B1", "C": "C1", "D": "D1", "E": "E1", "F": "F1", "G": "G1"},
                    agroecologie_place="Agroécologie moderne et intensive qui domine avec grandes superficies",
                    leviers=["Sécurisation foncière", "Formation des producteurs", "Schéma directeur appliqué"],
                    contraintes=["Pression urbaine", "Faible motivation des jeunes pour l'agriculture"],
                    points_basculement=["Équilibre entre besoins agricoles et urbains", "Renforcement du cadre juridique"]
                ),
                ScenarioProspectif(
                    nom="Baguineda en mouvement - Baguineda en transition", code="A3B1C1D2E2F2G1", zone=ZoneType.OPIB, type_scenario=TypeScenario.ALTERNATIF,
                    description="Transition vers l'agroécologie avec développement équilibré",
                    configuration={"A": "A3", "B": "B1", "C": "C1", "D": "D2", "E": "E2", "F": "F2", "G": "G1"},
                    agroecologie_place="Transition agroécologique en marche, agriculture conventionnelle dominante mais dynamique vers le bio",
                    leviers=["Projets de promotion agroécologique", "Formation en maraîchage bio"],
                    contraintes=["Méfiance des producteurs", "Faible valorisation commerciale des produits bio"],
                    points_basculement=["Amélioration accès aux intrants bio", "Connexion avec les marchés"]
                ),
                ScenarioProspectif(
                    nom="Baguineda tô bè - L'espoir est permis", code="A2B2C4D3E3F3G2", zone=ZoneType.OPIB, type_scenario=TypeScenario.DESIRABLE,
                    description="Développement local porté par l'agroécologie avec quelques défis",
                    configuration={"A": "A2", "B": "B2", "C": "C4", "D": "D3", "E": "E3", "F": "F3", "G": "G2"},
                    agroecologie_place="Agriculture dominée par l'agroécologie et biologique avec innovations techniques intensives",
                    leviers=["Loi d'orientation agricole appliquée", "Nouvelle dynamique de pratiques agroécologiques"],
                    contraintes=["Insuffisance des terres", "Faible disponibilité des intrants agroécologiques"],
                    points_basculement=["Priorisation de l'agriculture", "Mouvement social pour la protection des terres"]
                ),
                ScenarioProspectif(
                    nom="Baguineda pôle urbain", code="A4B2C1D5E2F5G3", zone=ZoneType.OPIB, type_scenario=TypeScenario.TENDANCIEL,
                    description="Urbanisation dominante au détriment de l'agriculture",
                    configuration={"A": "A4", "B": "B2", "C": "C1", "D": "D5", "E": "E2", "F": "F5", "G": "G3"},
                    agroecologie_place="Agroécologie de 'terrasses' sur petites superficies, agriculture menacée de disparition",
                    leviers=["Développement économique", "Infrastructures de communication"],
                    contraintes=["Disparition des terres agricoles", "Perte du potentiel agricole"],
                    points_basculement=["Plan d'utilisation des terres", "Développement de l'agriculture urbaine"]
                ),
                ScenarioProspectif(
                    nom="Baguineda baa kèdi - Que faire à Baguineda?", code="A4B3C2D4E4F4G5", zone=ZoneType.OPIB, type_scenario=TypeScenario.CRITIQUE,
                    description="Scénario critique avec dégradation générale des conditions",
                    configuration={"A": "A4", "B": "B3", "C": "C2", "D": "D4", "E": "E4", "F": "F4", "G": "G5"},
                    agroecologie_place="Quelques pratiques agroécologiques marginales, activité agricole menacée de disparition",
                    leviers=["Prise de conscience collective", "Appuis externes"],
                    contraintes=["Dégradation des infrastructures", "Rareté des terres", "Financement inaccessible"],
                    points_basculement=["Révision des priorités de développement", "Mobilisation communautaire"]
                )
            ]
            
            self.scenarios_par_zone[ZoneType.OPIB].extend(scenarios_opib)
            print(f"✅ {len(scenarios_opib)} scénarios OPIB chargés")
            
        except Exception as e:
            print(f"❌ Erreur lors du chargement des scénarios OPIB: {e}")

    def generer_scenarios_ohvn_par_miroir(self):
        """Génère des scénarios pour OHVN par effet miroir conceptuel avec OPIB"""
        try:
            scenarios_ohvn = [
                ScenarioProspectif(
                    nom="OHVN, Pivot Agro-Industriel", code="A5B1C2D1E2F2G1", zone=ZoneType.OHVN, type_scenario=TypeScenario.TENDANCIEL,
                    description="Spécialisation dans l'agro-industrie avec production intensive conventionnelle",
                    configuration={"A": "A5", "B": "B1", "C": "C2", "D": "D1", "E": "E2", "F": "F2", "G": "G1"},
                    agroecologie_place="Niches agroécologiques certifiées pour l'exportation, dominée par l'agriculture conventionnelle",
                    leviers=["Investissements privés", "Infrastructures de transformation", "Débouchés commerciaux"],
                    contraintes=["Pression sur les ressources", "Dépendance aux intrants externes"],
                    points_basculement=["Gestion durable des ressources", "Diversification des modèles agricoles"]
                ),
                ScenarioProspectif(
                    nom="OHVN, Écosystème Résilient et Agroécologique", code="A2B2C4D2E3F1G2", zone=ZoneType.OHVN, type_scenario=TypeScenario.DESIRABLE,
                    description="Territoire résilient basé sur l'agroécologie et les circuits courts",
                    configuration={"A": "A2", "B": "B2", "C": "C4", "D": "D2", "E": "E3", "F": "F1", "G": "G2"},
                    agroecologie_place="Agroécologie structurante avec pratiques diversifiées et biologiques pour l'autosuffisance",
                    leviers=["Organisations paysannes dynamiques", "Circuits courts", "Savoir-faire locaux"],
                    contraintes=["Accès limité aux marchés", "Faible mécanisation"],
                    points_basculement=["Valorisation des produits locaux", "Renforcement des capacités techniques"]
                ),
                ScenarioProspectif(
                    nom="OHVN, Laboratoire de l'Agriculture de Précision", code="A1B1C1D1E1F1G1", zone=ZoneType.OHVN, type_scenario=TypeScenario.ALTERNATIF,
                    description="Innovation technologique de pointe pour une agriculture high-tech",
                    configuration={"A": "A1", "B": "B1", "C": "C1", "D": "D1", "E": "E1", "F": "F1", "G": "G1"},
                    agroecologie_place="Agroécologie de précision avec drones, capteurs et optimisation des intrants",
                    leviers=["Recherche et développement", "Partenariats technologiques", "Formation spécialisée"],
                    contraintes=["Fracture numérique", "Coûts d'investissement élevés"],
                    points_basculement=["Accès au financement innovation", "Adaptation des technologies au contexte local"]
                ),
                ScenarioProspectif(
                    nom="OHVN, Corridor Économique et Logistique", code="A4B1C1D5E2F5G3", zone=ZoneType.OHVN, type_scenario=TypeScenario.TENDANCIEL,
                    description="Développement logistique dominant au détriment de la fonction agricole",
                    configuration={"A": "A4", "B": "B1", "C": "C1", "D": "D5", "E": "E2", "F": "F5", "G": "G3"},
                    agroecologie_place="Agriculture résiduelle et périurbaine, agroécologie confidentielle",
                    leviers=["Position géostratégique", "Développement des infrastructures", "Création d'emplois non agricoles"],
                    contraintes=["Concurrence pour l'espace", "Dépendance alimentaire externe"],
                    points_basculement=["Planification territoriale intégrée", "Développement de corridors écologiques"]
                ),
                ScenarioProspectif(
                    nom="OHVN, Territoires Fragmentés", code="A4B3C3D4E5F4G5", zone=ZoneType.OHVN, type_scenario=TypeScenario.CRITIQUE,
                    description="Fragmentation et tensions territoriales avec dégradation des conditions",
                    configuration={"A": "A4", "B": "B3", "C": "C3", "D": "D4", "E": "E5", "F": "F4", "G": "G5"},
                    agroecologie_place="Pratiques agroécologiques de survie, agriculture en difficulté",
                    leviers=["Résilience communautaire", "Initiatives locales"],
                    contraintes=["Conflits fonciers", "Dégradation des infrastructures", "Insécurité alimentaire"],
                    points_basculement=["Médiation territoriale", "Reconstruction de la gouvernance locale"]
                )
            ]
            
            self.scenarios_par_zone[ZoneType.OHVN].extend(scenarios_ohvn)
            print(f"✅ {len(scenarios_ohvn)} scénarios OHVN générés par effet miroir")
            
        except Exception as e:
            print(f"❌ Erreur lors de la génération des scénarios OHVN: {e}")

    def _define_forces_motrices(self) -> Dict[str, ForceMotrice]:
        """Définit les forces motrices complètes."""
        forces_data = [
            {"code": "A", "nom": "Formes de production agricole", "definition": "Nature et diversité des pratiques et formes d'agriculture",
             "etats_futurs": {"A1": "Toutes formes d'agriculture présentes et performantes avec hautes technologies", "A2": "Agriculture dominée par l'agroécologie et biologique avec innovations techniques", "A3": "Pratiques conventionnelles avec transition vers agroécologie intensive", "A4": "Pratiques conventionnelles multiples peu performantes, agroécologie marginale", "A5": "Productions intensives conventionnelles avec hautes technologies"}},
            {"code": "B", "nom": "Communication", "definition": "État des infrastructures de communication et télécommunications",
             "etats_futurs": {"B1": "Infrastructures hautes technologies de bonne qualité dans toute la commune", "B2": "Infrastructures de bonne qualité seulement dans certaines parties", "B3": "Infrastructures existantes mais de mauvaise qualité", "B4": "Réseau routier presque disparu, télécommunications inexistantes"}},
            {"code": "C", "nom": "Potentiel ressource minière", "definition": "État du potentiel minier et de son exploitation",
             "etats_futurs": {"C1": "Ressources découvertes, exploitées et transformées sur place avec gestion rigoureuse", "C2": "Ressources exploitées mais transformées en dehors de la commune", "C3": "Ressources mal exploitées et mal gérées", "C4": "Ressources non exploitées sur place"}},
            {"code": "D", "nom": "Production et accès à l'alimentation", "definition": "Quantité et qualité de la production alimentaire et accès à une bonne alimentation",
             "etats_futurs": {"D1": "Baguineda est un agropôle, production abondante et accessible avec excédents", "D2": "Production disponible pour autosuffisance mais pas de vente vers l'extérieur", "D3": "Surproduction avec alimentation en excès et mévente", "D4": "Production déficitaire, accès assuré par importation de mauvaise qualité", "D5": "Production locale très faible, alimentation assurée exclusivement de l'extérieur"}},
            {"code": "E", "nom": "Financement du développement", "definition": "Disponibilité des ressources financières pour les politiques locales",
             "etats_futurs": {"E1": "Ressources disponibles et accessibles pour tous les secteurs", "E2": "Ressources disponibles seulement pour quelques secteurs prioritaires", "E3": "Ressources disponibles mais insuffisantes", "E4": "Ressources disponibles mais inaccessibles à cause des procédures", "E5": "Ressources ni disponibles ni accessibles"}},
            {"code": "F", "nom": "Aménagements hydroagricoles et accès à l'eau", "definition": "État des aménagements hydroagricoles, infrastructures et accès à l'eau",
             "etats_futurs": {"F1": "Aménagements de bonne qualité avec technologies de pointe, gestion concertée", "F2": "Aménagements disponibles mais gestion assurée par entités particulières", "F3": "Aménagements présents mais défectueux et mal gérés", "F4": "Aménagements totalement dégradés, eau inaccessible", "F5": "Aménagements n'existent plus, zone occupée par habitat"}},
            {"code": "G", "nom": "Accès à la terre", "definition": "Disponibilité et accès aux ressources foncières",
             "etats_futurs": {"G1": "Terres disponibles, accessibles et sécurisées pour agriculture et urbanisation", "G2": "Terres disponibles pour agriculture sans tenir compte du développement urbain", "G3": "Terres prioritairement orientées vers l'urbanisation, pas pour agriculture", "G4": "Terres rares et inaccessibles pour agriculture et urbanisation", "G5": "Terres rares et difficiles d'accès pour tous usages"}}
        ]
        
        forces = {}
        for force_data in forces_data:
            force = ForceMotrice(**force_data)
            forces[force.code] = force
        return forces

    def _define_scenarios_prospectifs(self) -> Dict[str, ScenarioProspectif]:
        """Définit l'ensemble complet des scénarios prospectifs pour OPIB et OHVN."""
        scenarios = {}
        
        if self.zone_name.upper() == "OPIB":
            scenarios = {
                "actuel": ScenarioProspectif(nom="Statut Quo OPIB", code="A4B2C4D4E3F3G5", zone=ZoneType.OPIB, type_scenario=TypeScenario.TENDANCIEL, description="Situation actuelle à Baguineda avec tendances existantes", configuration={"A": "A4", "B": "B2", "C": "C4", "D": "D4", "E": "E3", "F": "F3", "G": "G5"}, agroecologie_place="Pratiques conventionnelles dominantes, transition lente vers l'agroécologie", leviers=["Maintenance des infrastructures", "Appuis techniques existants"], contraintes=["Dégradation des sols", "Pression foncière", "Financement limité"], points_basculement=["Plan de développement territorial", "Renforcement des OP"]),
                "baguineda_gnogon_te": ScenarioProspectif(nom="Baguineda gnogon tè - Baguineda est le meilleur", code="A1B1C1D1E1F1G1", zone=ZoneType.OPIB, type_scenario=TypeScenario.DESIRABLE, description="Scénario idéal avec toutes les forces motrices dans leur état optimal", configuration={"A": "A1", "B": "B1", "C": "C1", "D": "D1", "E": "E1", "F": "F1", "G": "G1"}, agroecologie_place="Agroécologie moderne et intensive, grandes superficies optimisées", leviers=["Sécurisation foncière", "Formation des producteurs", "Schéma directeur appliqué"], contraintes=["Pression urbaine", "Faible motivation des jeunes pour l'agriculture"], points_basculement=["Équilibre entre besoins agricoles et urbains", "Renforcement du cadre juridique"]),
                "baguineda_en_mouvement": ScenarioProspectif(nom="Baguineda en mouvement - Baguineda en transition", code="A3B1C1D2E2F2G1", zone=ZoneType.OPIB, type_scenario=TypeScenario.ALTERNATIF, description="Transition vers l'agroécologie avec développement équilibré", configuration={"A": "A3", "B": "B1", "C": "C1", "D": "D2", "E": "E2", "F": "F2", "G": "G1"}, agroecologie_place="Transition agroécologique en marche, agriculture conventionnelle dominante mais dynamique", leviers=["Projets de promotion agroécologique", "Formation en maraîchage bio"], contraintes=["Méfiance des producteurs", "Faible valorisation commerciale des produits bio"], points_basculement=["Amélioration accès aux intrants bio", "Connexion avec les marchés"]),
                "baguineda_to_be": ScenarioProspectif(nom="Baguineda tô bè - L'espoir est permis", code="A2B2C4D3E3F3G2", zone=ZoneType.OPIB, type_scenario=TypeScenario.DESIRABLE, description="Développement local porté par l'agroécologie avec quelques défis", configuration={"A": "A2", "B": "B2", "C": "C4", "D": "D3", "E": "E3", "F": "F3", "G": "G2"}, agroecologie_place="Agriculture dominée par l'agroécologie et biologique, innovations techniques intensives", leviers=["Loi d'orientation agricole appliquée", "Nouvelle dynamique de pratiques agroécologiques"], contraintes=["Insuffisance des terres", "Faible disponibilité des intrants agroécologiques"], points_basculement=["Priorisation de l'agriculture", "Mouvement social pour la protection des terres"]),
                "baguineda_pole_urbain": ScenarioProspectif(nom="Baguineda pôle urbain", code="A4B2C1D5E2F5G3", zone=ZoneType.OPIB, type_scenario=TypeScenario.TENDANCIEL, description="Urbanisation dominante au détriment de l'agriculture", configuration={"A": "A4", "B": "B2", "C": "C1", "D": "D5", "E": "E2", "F": "F5", "G": "G3"}, agroecologie_place="Agroécologie marginale, agriculture menacée de disparition", leviers=["Développement économique", "Infrastructures de communication"], contraintes=["Disparition des terres agricoles", "Perte du potentiel agricole"], points_basculement=["Plan d'utilisation des terres", "Développement de l'agriculture urbaine"]),
                "baguineda_baa_kedi": ScenarioProspectif(nom="Baguineda baa kèdi - Que faire à Baguineda?", code="A4B3C2D4E4F4G5", zone=ZoneType.OPIB, type_scenario=TypeScenario.CRITIQUE, description="Scénario critique avec dégradation générale des conditions", configuration={"A": "A4", "B": "B3", "C": "C2", "D": "D4", "E": "E4", "F": "F4", "G": "G5"}, agroecologie_place="Pratiques agroécologiques marginales, activité agricole menacée de disparition", leviers=["Prise de conscience collective", "Appuis externes"], contraintes=["Dégradation des infrastructures", "Rareté des terres", "Financement inaccessible"], points_basculement=["Révision des priorités de développement", "Mobilisation communautaire"])
            }
        else:
            scenarios = {
                "actuel": ScenarioProspectif(nom="Statut Quo OHVN", code="A4B2C4D4E3F3G4", zone=ZoneType.OHVN, type_scenario=TypeScenario.TENDANCIEL, description="Situation actuelle à Tièlè avec spécificités locales", configuration={"A": "A4", "B": "B2", "C": "C4", "D": "D4", "E": "E3", "F": "F3", "G": "G4"}, agroecologie_place="Agriculture conventionnelle dominante", leviers=["Ressources en eau", "Savoir-faire local", "Proximité des marchés"], contraintes=["Sécheresse récurrente", "Enclavement partiel", "Accès limité aux intrants"], points_basculement=["Aménagements hydroagricoles", "Désenclavement"]),
                "ohvn_pivot_agroindustriel": ScenarioProspectif(nom="OHVN, Pivot Agro-Industriel", code="A5B1C2D1E2F2G1", zone=ZoneType.OHVN, type_scenario=TypeScenario.TENDANCIEL, description="Spécialisation dans l'agro-industrie avec production intensive conventionnelle", configuration={"A": "A5", "B": "B1", "C": "C2", "D": "D1", "E": "E2", "F": "F2", "G": "G1"}, agroecologie_place="Niches agroécologiques certifiées pour l'exportation, agriculture conventionnelle dominante", leviers=["Investissements privés", "Infrastructures de transformation", "Débouchés commerciaux"], contraintes=["Pression sur les ressources", "Dépendance aux intrants externes"], points_basculement=["Gestion durable des ressources", "Diversification des modèles agricoles"]),
                "ohvn_ecosysteme_resilient": ScenarioProspectif(nom="OHVN, Écosystème Résilient et Agroécologique", code="A2B2C4D2E3F1G2", zone=ZoneType.OHVN, type_scenario=TypeScenario.DESIRABLE, description="Territoire résilient basé sur l'agroécologie et les circuits courts", configuration={"A": "A2", "B": "B2", "C": "C4", "D": "D2", "E": "E3", "F": "F1", "G": "G2"}, agroecologie_place="Agroécologie structurante avec pratiques diversifiées et biologiques pour l'autosuffisance", leviers=["Organisations paysannes dynamiques", "Circuits courts", "Savoir-faire locaux"], contraintes=["Accès limité aux marchés", "Faible mécanisation"], points_basculement=["Valorisation des produits locaux", "Renforcement des capacités techniques"]),
                "ohvn_agriculture_precision": ScenarioProspectif(nom="OHVN, Laboratoire de l'Agriculture de Précision", code="A1B1C1D1E1F1G1", zone=ZoneType.OHVN, type_scenario=TypeScenario.ALTERNATIF, description="Innovation technologique de pointe pour une agriculture high-tech", configuration={"A": "A1", "B": "B1", "C": "C1", "D": "D1", "E": "E1", "F": "F1", "G": "G1"}, agroecologie_place="Agroécologie de précision avec drones, capteurs et optimisation des intrants", leviers=["Recherche et développement", "Partenariats technologiques", "Formation spécialisée"], contraintes=["Fracture numérique", "Coûts d'investissement élevés"], points_basculement=["Accès au financement innovation", "Adaptation des technologies au contexte local"]),
                "ohvn_corridor_economique": ScenarioProspectif(nom="OHVN, Corridor Économique et Logistique", code="A4B1C1D5E2F5G3", zone=ZoneType.OHVN, type_scenario=TypeScenario.TENDANCIEL, description="Développement logistique dominant au détriment de la fonction agricole", configuration={"A": "A4", "B": "B1", "C": "C1", "D": "D5", "E": "E2", "F": "F5", "G": "G3"}, agroecologie_place="Agriculture résiduelle et périurbaine, agroécologie confidentielle", leviers=["Position géostratégique", "Développement des infrastructures", "Création d'emplois non agricoles"], contraintes=["Concurrence pour l'espace", "Dépendance alimentaire externe"], points_basculement=["Planification territoriale intégrée", "Développement de corridors écologiques"]),
                "ohvn_territoires_fragmentes": ScenarioProspectif(nom="OHVN, Territoires Fragmentés", code="A4B3C3D4E5F4G5", zone=ZoneType.OHVN, type_scenario=TypeScenario.CRITIQUE, description="Fragmentation et tensions territoriales avec dégradation des conditions", configuration={"A": "A4", "B": "B3", "C": "C3", "D": "D4", "E": "E5", "F": "F4", "G": "G5"}, agroecologie_place="Pratiques agroécologiques de survie, agriculture en difficulté", leviers=["Résilience communautaire", "Initiatives locales"], contraintes=["Conflits fonciers", "Dégradation des infrastructures", "Insécurité alimentaire"], points_basculement=["Médiation territoriale", "Reconstruction de la gouvernance locale"])
            }
        return scenarios

    def _get_scenario_prospectif(self, scenario_type: str) -> ScenarioProspectif:
        """Retourne le scénario prospectif correspondant."""
        return self.scenarios_prospectifs.get(scenario_type, list(self.scenarios_prospectifs.values())[0])

    def _define_scenario_effects_from_prospective(self) -> Dict[str, Any]:
        """Définit les effets du scénario basés sur la configuration prospective."""
        try:
            scenario = self.scenario_prospectif
            config = scenario.configuration
            
            base_effects = {
                "policy": self._calculate_policy_effect_from_config(config),
                "climate": self._calculate_climate_effect_from_config(config),
                "adoption_boost": self._calculate_adoption_boost_from_config(config),
                "regression_effects": self._calculate_regression_effects(config),
                "territorial_effects": self._calculate_territorial_effects(config),
                "description": scenario.description,
                "scenario_prospectif": scenario
            }
            return base_effects
        except Exception as e:
            print(f"⚠️ Erreur dans les effets du scénario: {e}")
            return self._get_default_scenario_effects()

    def _get_default_scenario_effects(self) -> Dict[str, Any]:
        return {"policy": "neutral", "climate": 1.0, "adoption_boost": 0.0, "regression_effects": {}, "territorial_effects": {}, "description": "Scénario par défaut"}

    def _calculate_policy_effect_from_config(self, config: Dict[str, str]) -> str:
        if config.get("E") in ["E1", "E2"]: return "highly_supportive"
        elif config.get("E") in ["E3", "E4"]: return "supportive"
        else: return "neutral"

    def _calculate_climate_effect_from_config(self, config: Dict[str, str]) -> float:
        if config.get("F") in ["F1", "F2"]: return 1.2
        elif config.get("F") in ["F4", "F5"]: return 0.8
        else: return 1.0

    def _calculate_adoption_boost_from_config(self, config: Dict[str, str]) -> float:
        boost = 0.0
        if config.get("A") in ["A1", "A2"]: boost += 0.3
        if config.get("G") in ["G1", "G2"]: boost += 0.2
        return boost

    def _calculate_regression_effects(self, config: Dict[str, str]) -> Dict[str, float]:
        effects = {}
        if config.get("D") in ["D4", "D5"]: effects["food_security"] = 0.7
        if config.get("B") in ["B3", "B4"]: effects["market_access"] = 0.6
        return effects

    def _calculate_territorial_effects(self, config: Dict[str, str]) -> Dict[str, float]:
        effects = {}
        if config.get("C") in ["C1", "C2"]: effects["mining_development"] = 1.3
        if config.get("F") in ["F1", "F2"]: effects["water_management"] = 1.4
        return effects

    def _prepare_agent_data(self, agent_data: pd.DataFrame):
        """Prépare les données d'agents de manière robuste."""
        try:
            if agent_data is None or len(agent_data) == 0:
                print(f"⚠️ Aucune donnée d'agents fournie pour {self.zone_name}")
                self.agent_data = self._create_default_agent_data()
                return
                
            if 'zone' in agent_data.columns:
                self.agent_data = agent_data[agent_data['zone'] == self.zone_name].copy()
            else:
                self.agent_data = agent_data.copy()
                print(f"⚠️ Colonne 'zone' non trouvée, utilisation de toutes les données")
            
            if len(self.agent_data) == 0:
                print(f"⚠️ Aucune donnée pour {self.zone_name}, création d'agents par défaut")
                self.agent_data = self._create_default_agent_data()
            elif len(self.agent_data) < self.num_agents:
                print(f"⚠️ Données insuffisantes ({len(self.agent_data)}), utilisation de tous les agents disponibles")
                self.num_agents = len(self.agent_data)
                
        except Exception as e:
            print(f"❌ Erreur lors de la préparation des données: {e}")
            self.agent_data = self._create_default_agent_data()

    def _create_default_agent_data(self) -> pd.DataFrame:
        """Crée des données d'agents par défaut."""
        np.random.seed(42)
        n_agents = min(self.num_agents, 30)
        data = []
        for i in range(n_agents):
            agent = {
                'zone': self.zone_name,
                'caet_tot': np.random.normal(40 if self.zone_name == "OPIB" else 35, 10),
                'farm_revenue': np.random.uniform(500000, 2000000),
                'soil_health': np.random.normal(55 if self.zone_name == "OPIB" else 50, 15),
                'adopts_agroecology': np.random.random() < 0.2,
            }
            data.append(agent)
        return pd.DataFrame(data)

    def _initialize_agents(self):
        """Initialise la population d'agents de manière robuste."""
        try:
            if len(self.agent_data) > 0:
                if len(self.agent_data) >= self.num_agents:
                    sample_data = self.agent_data.sample(n=self.num_agents, random_state=42, replace=False)
                else:
                    sample_data = self.agent_data.sample(n=self.num_agents, random_state=42, replace=True)
            else:
                sample_data = self.agent_data
                
            self.all_agents = []
            for i, (_, row) in enumerate(sample_data.iterrows()):
                try:
                    adapted_row = self._adapt_agent_to_scenario(row)
                    self.all_agents.append(ExploitationTAPE(i, self, adapted_row))
                except Exception as e:
                    print(f"⚠️ Erreur création agent {i}: {e}")
                    continue
                    
            if not self.all_agents:
                print("❌ Aucun agent créé, création d'agents de secours")
                self._create_fallback_agents()
                
        except Exception as e:
            print(f"❌ Erreur lors de l'initialisation des agents: {e}")
            self._create_fallback_agents()

    def _create_fallback_agents(self):
        """Crée des agents de secours en cas d'erreur."""
        self.all_agents = []
        for i in range(min(10, self.num_agents)):
            default_data = pd.Series({
                'zone': self.zone_name, 'caet_tot': 40.0, 'farm_revenue': 1000000, 'soil_health': 50.0, 'adopts_agroecology': False
            })
            self.all_agents.append(ExploitationTAPE(i, self, default_data))

    def _adapt_agent_to_scenario(self, agent_data: pd.Series) -> pd.Series:
        """Adapte les données d'un agent au scénario."""
        try:
            scenario = self.scenario_prospectif
            
            if scenario.type_scenario == TypeScenario.DESIRABLE:
                multiplier = 1.05
                adoption_boost = 0.1
            elif scenario.type_scenario == TypeScenario.CRITIQUE:
                multiplier = 0.95
                adoption_boost = -0.1
            else:
                multiplier = 1.0
                adoption_boost = 0.0
                
            for indicator in self.PERFORMANCE_INDICATORS.keys():
                if indicator in agent_data.index and indicator != 'pest_score':
                    agent_data[indicator] = agent_data[indicator] * multiplier
                elif indicator in agent_data.index and indicator == 'pest_score':
                    agent_data[indicator] = agent_data[indicator] * (2 - multiplier)
                    
            if 'adopts_agroecology' in agent_data.index:
                current_prob = agent_data['adopts_agroecology'] if isinstance(agent_data['adopts_agroecology'], (int, float)) else 0.2
                agent_data['adopts_agroecology'] = current_prob + adoption_boost
                    
            return agent_data
        except Exception as e:
            print(f"⚠️ Erreur adaptation agent au scénario: {e}")
            return agent_data

    def _analyze_agent_situations(self):
        """Analyse la situation de chaque agent pour personnaliser les effets de scénario."""
        self.agent_situations = {}
        self.zone_situation_profile = {
            'vulnerable_agents': 0, 'emerging_agents': 0, 'consolidated_agents': 0, 'advanced_agents': 0,
            'avg_technology_level': 0, 'avg_family_labor': 0, 'adoption_potential': 0
        }
        
        for agent in self.all_agents:
            situation = self._analyze_individual_agent_situation(agent)
            self.agent_situations[agent.unique_id] = situation
            
            if situation['category'] == 'vulnerable': self.zone_situation_profile['vulnerable_agents'] += 1
            elif situation['category'] == 'emerging': self.zone_situation_profile['emerging_agents'] += 1
            elif situation['category'] == 'consolidated': self.zone_situation_profile['consolidated_agents'] += 1
            elif situation['category'] == 'advanced': self.zone_situation_profile['advanced_agents'] += 1
                
            self.zone_situation_profile['avg_technology_level'] += situation['technology_level']
            self.zone_situation_profile['avg_family_labor'] += situation['family_labor_score']
            self.zone_situation_profile['adoption_potential'] += situation['adoption_potential']
        
        total_agents = len(self.all_agents)
        if total_agents > 0:
            self.zone_situation_profile['avg_technology_level'] /= total_agents
            self.zone_situation_profile['avg_family_labor'] /= total_agents
            self.zone_situation_profile['adoption_potential'] /= total_agents

    def _analyze_individual_agent_situation(self, agent) -> Dict[str, Any]:
        """Analyse la situation individuelle d'un agent."""
        tech_level = agent._calculate_technology_adoption_level()
        ae_score = agent._calculate_agroecology_score()
        family_labor = agent._calculate_family_labor_score()
        economic_score = self._calculate_economic_situation(agent)
        resilience_score = self._calculate_resilience_situation(agent)
        category = self._categorize_agent_situation(tech_level, ae_score, economic_score, resilience_score)
        adoption_potential = self._calculate_personalized_adoption_potential(agent, category, tech_level, economic_score)
        
        return {
            'category': category, 'technology_level': tech_level, 'agroecology_score': ae_score,
            'family_labor_score': family_labor, 'economic_score': economic_score, 'resilience_score': resilience_score,
            'adoption_potential': adoption_potential, 'specific_needs': self._identify_specific_needs(agent, category),
            'strengths': self._identify_strengths(agent, category), 'vulnerabilities': self._identify_vulnerabilities(agent, category)
        }

    def _calculate_economic_situation(self, agent) -> float:
        economic_indicators = []
        if hasattr(agent, 'farm_revenue'):
            revenue_score = min(100, agent.farm_revenue / 50000)
            economic_indicators.append(revenue_score * 0.4)
        if hasattr(agent, 'profit_margin'):
            economic_indicators.append(agent.profit_margin * 0.3)
        if hasattr(agent, 'value_added'):
            economic_indicators.append(min(100, agent.value_added) * 0.3)
        return np.mean(economic_indicators) if economic_indicators else 30.0

    def _calculate_resilience_situation(self, agent) -> float:
        resilience_indicators = []
        if hasattr(agent, 'soil_health'): resilience_indicators.append(agent.soil_health * 0.25)
        if hasattr(agent, 'biodiversity_index'): resilience_indicators.append(agent.biodiversity_index * 0.25)
        if hasattr(agent, 'ecological_resilience'): resilience_indicators.append(agent.ecological_resilience * 0.25)
        diversity_score = len([tech for tech, level in agent.ae_technologies.items() if level >= 2]) / len(agent.ae_technologies) * 100
        resilience_indicators.append(diversity_score * 0.25)
        return np.mean(resilience_indicators) if resilience_indicators else 40.0

    def _categorize_agent_situation(self, tech_level: float, ae_score: float, economic_score: float, resilience_score: float) -> str:
        overall_score = (tech_level * 0.2 + ae_score * 0.3 + economic_score * 0.3 + resilience_score * 0.2)
        if overall_score >= 70: return "advanced"
        elif overall_score >= 50: return "consolidated"
        elif overall_score >= 30: return "emerging"
        else: return "vulnerable"

    def _calculate_personalized_adoption_potential(self, agent, category: str, tech_level: float, economic_score: float) -> float:
        base_potential = 0.0
        if category == "vulnerable": base_potential = 0.1 + (tech_level / 100) * 0.2
        elif category == "emerging": base_potential = 0.3 + (tech_level / 100) * 0.3
        elif category == "consolidated": base_potential = 0.5 + (tech_level / 100) * 0.4
        elif category == "advanced": base_potential = 0.7 + (tech_level / 100) * 0.5
        
        if category != "vulnerable": base_potential += (economic_score / 100) * 0.2
        if hasattr(agent, 'innovation_capacity'): base_potential += agent.innovation_capacity * 0.15
        if hasattr(agent, 'social_network'): base_potential += agent.social_network * 0.10
        return min(0.95, base_potential)

    def _identify_specific_needs(self, agent, category: str) -> List[str]:
        needs = []
        if category == "vulnerable": needs.extend(["Accès aux intrants de base", "Formation technique élémentaire", "Soutien alimentaire d'urgence", "Accès au crédit solidaire"])
        elif category == "emerging": needs.extend(["Formation agroécologique", "Accès aux marchés locaux", "Infrastructures de base", "Conseil technique régulier"])
        elif category == "consolidated": needs.extend(["Technologies avancées", "Accès aux marchés régionaux", "Formation gestion d'entreprise", "Réseautage professionnel"])
        elif category == "advanced": needs.extend(["Innovation technologique", "Accès aux marchés d'exportation", "Formation leadership", "Participation aux politiques"])
        
        if hasattr(agent, 'farm_area') and agent.farm_area < 2: needs.append("Intensification durable")
        if hasattr(agent, 'soil_health') and agent.soil_health < 40: needs.append("Régénération des sols")
        return needs

    def _identify_strengths(self, agent, category: str) -> List[str]:
        strengths = []
        if hasattr(agent, 'family_labor_score') and agent.family_labor_score > 60: strengths.append("Main d'œuvre familiale importante")
        tech_level = agent._calculate_technology_adoption_level()
        if tech_level > 50: strengths.append("Niveau technologique correct")
        if tech_level > 70: strengths.append("Haute compétence technique")
        if hasattr(agent, 'farm_revenue') and agent.farm_revenue > 1500000: strengths.append("Revenus stables")
        if hasattr(agent, 'social_network') and agent.social_network > 0.6: strengths.append("Réseau social développé")
        return strengths

    def _identify_vulnerabilities(self, agent, category: str) -> List[str]:
        vulnerabilities = []
        if hasattr(agent, 'farm_revenue') and agent.farm_revenue < 500000: vulnerabilities.append("Faibles revenus")
        if hasattr(agent, 'profit_margin') and agent.profit_margin < 15: vulnerabilities.append("Faible marge bénéficiaire")
        tech_level = agent._calculate_technology_adoption_level()
        if tech_level < 30: vulnerabilities.append("Faible niveau technologique")
        if hasattr(agent, 'soil_health') and agent.soil_health < 40: vulnerabilities.append("Sol dégradé")
        if hasattr(agent, 'water_management') and agent.water_management < 40: vulnerabilities.append("Gestion de l'eau insuffisante")
        if hasattr(agent, 'family_labor_score') and agent.family_labor_score < 30: vulnerabilities.append("Main d'œuvre familiale limitée")
        return vulnerabilities

    def _get_agent_situation_summary(self) -> str:
        if not hasattr(self, 'zone_situation_profile'): return "Non analysé"
        profile = self.zone_situation_profile
        total = len(self.all_agents)
        return (f"Vulnérables: {profile['vulnerable_agents']}/{total} | Émergents: {profile['emerging_agents']}/{total} | Consolidés: {profile['consolidated_agents']}/{total} | Avancés: {profile['advanced_agents']}/{total} | Tech moyenne: {profile['avg_technology_level']:.1f}%")

    def _initialize_tracking_systems(self):
        self.model_history = []
        self.agent_history = []
        self.prospective_history = []
        self.successful_steps = 0
        self.total_steps = 0

    def run_model(self, steps: int = 5) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        print(f"   🎮 Simulation {self.zone_name} - {self.scenario_type} ({steps} steps)")
        model_data = []
        agent_data = []
        prospective_data = []
        
        try:
            for step in range(steps + 1):
                current_year = self.start_year + step
                for idx, agent in enumerate(self.all_agents):
                    try:
                        agent_effects = self._calculate_prospective_agent_effects(agent)
                        agent.step(agent_effects)
                        agent_row = agent.get_agent_data()
                        agent_row['year'] = current_year
                        agent_row['step'] = step
                        agent_row['scenario_type'] = self.scenario_type
                        agent_data.append(agent_row)
                        
                        model_entry = {
                            'year': current_year, 'zone': self.zone_name, 'scenario_type': self.scenario_type,
                            'agent_id': f"{self.zone_name}_{idx}", 'adopts_agroecology': agent_row.get('adopts_agroecology', False),
                            'performance_globale': agent_row.get('caet_tot', 0) / 100, 'resilience': np.random.normal(60, 15) * self._get_scenario_multiplier() / 100
                        }
                        
                        for indicator in self.PERFORMANCE_INDICATORS.keys():
                            if indicator in agent_row: model_entry[indicator] = agent_row[indicator]
                        model_data.append(model_entry)
                    except Exception as e:
                        print(f"⚠️ Erreur agent {idx} step {step}: {e}")
                        continue
                
                viability = self._calculate_scenario_viability(step, steps)
                coherence = self._calculate_territorial_coherence(step, steps)
                implementation = self._calculate_agroecology_implementation(step, steps)
                
                prospective_data.append({
                    'year': current_year, 'zone': self.zone_name, 'scenario_type': self.scenario_type,
                    'scenario_viability': viability, 'territorial_coherence': coherence, 'agroecology_implementation': implementation,
                    'scenario_name': self.scenario_prospectif.nom, 'scenario_code': self.scenario_prospectif.code
                })
                
                self.current_year += 1
                self.current_step += 1
                self.successful_steps += 1
            self.total_steps = steps
        except Exception as e:
            print(f"❌ Erreur lors de l'exécution du modèle: {e}")
        return pd.DataFrame(model_data), pd.DataFrame(agent_data), pd.DataFrame(prospective_data)

    def _calculate_prospective_agent_effects(self, agent) -> Dict[str, Any]:
        base_effects = self.scenario_effects.copy()
        agent_situation = self.agent_situations.get(agent.unique_id, {})
        if agent_situation:
            category = agent_situation.get('category', 'emerging')
            base_effects["personalized_effects"] = self._get_personalized_effects(category, agent_situation)
            base_effects["strength_bonus"] = self._calculate_strength_bonus(agent_situation)
            base_effects["vulnerability_penalty"] = self._calculate_vulnerability_penalty(agent_situation)
            base_effects["personal_adoption_potential"] = agent_situation.get('adoption_potential', 0.3)
        if hasattr(agent, 'innovation_capacity'): base_effects["innovation_bonus"] = agent.innovation_capacity * 0.15
        if hasattr(agent, 'social_network'): base_effects["network_effect"] = agent.social_network * 0.12
        if hasattr(agent, 'risk_aversion'): base_effects["risk_effect"] = 1.0 - (agent.risk_aversion * 0.2)
        return base_effects

    def _get_personalized_effects(self, category: str, situation: Dict) -> Dict[str, float]:
        if category == "vulnerable": return {"support_intensity": 1.5, "technology_absorption": 0.7, "risk_tolerance": 0.5, "training_priority": 1.8}
        elif category == "emerging": return {"support_intensity": 1.2, "technology_absorption": 0.9, "risk_tolerance": 0.7, "training_priority": 1.3}
        elif category == "consolidated": return {"support_intensity": 1.0, "technology_absorption": 1.2, "risk_tolerance": 0.9, "training_priority": 1.0}
        elif category == "advanced": return {"support_intensity": 0.8, "technology_absorption": 1.5, "risk_tolerance": 1.2, "training_priority": 0.7, "demonstration_effect": 1.3}
        return {}

    def _calculate_strength_bonus(self, situation: Dict) -> float:
        bonus = 0.0
        if situation.get('technology_level', 0) > 70: bonus += 0.2
        if situation.get('family_labor_score', 0) > 60: bonus += 0.15
        if situation.get('economic_score', 0) > 60: bonus += 0.15
        if situation.get('resilience_score', 0) > 70: bonus += 0.1
        return min(0.5, bonus)

    def _calculate_vulnerability_penalty(self, situation: Dict) -> float:
        penalty = 0.0
        if situation.get('technology_level', 0) < 30: penalty += 0.2
        if situation.get('economic_score', 0) < 30: penalty += 0.2
        if situation.get('resilience_score', 0) < 30: penalty += 0.15
        return min(0.5, penalty)

    def _get_scenario_multiplier(self): return self.SCENARIO_MULTIPLIERS.get(self.scenario_type, 1.0)

    def _calculate_scenario_viability(self, current_step, total_steps):
        progress = current_step / total_steps
        multiplier = self._get_scenario_multiplier()
        if self.scenario_prospectif.type_scenario == TypeScenario.DESIRABLE: base_viability = 0.5 + (0.4 * progress * multiplier)
        elif self.scenario_prospectif.type_scenario == TypeScenario.CRITIQUE: base_viability = 0.2 + (0.3 * progress * multiplier)
        else: base_viability = 0.3 + (0.5 * progress * multiplier)
        return min(0.95, base_viability + np.random.normal(0, 0.05))

    def _calculate_territorial_coherence(self, current_step, total_steps):
        progress = current_step / total_steps
        multiplier = self._get_scenario_multiplier()
        base_coherence = 0.4 + (0.4 * progress * multiplier)
        return min(0.9, base_coherence + np.random.normal(0, 0.03))

    def _calculate_agroecology_implementation(self, current_step, total_steps):
        progress = current_step / total_steps
        multiplier = self._get_scenario_multiplier()
        adoption_rate = np.mean([agent.adopts_agroecology for agent in self.all_agents]) if self.all_agents else 0.2
        base_implementation = adoption_rate + (0.6 * progress * multiplier)
        return min(0.95, base_implementation + np.random.normal(0, 0.04))

    def get_detailed_agent_analysis(self, agent_id: int) -> Dict[str, Any]:
        if agent_id not in self.agent_situations: return {"error": "Agent non trouvé"}
        agent = self.all_agents[agent_id]
        situation = self.agent_situations[agent_id]
        return {
            'agent_id': agent_id, 'category': situation['category'], 'technology_level': situation['technology_level'],
            'agroecology_score': situation['agroecology_score'], 'economic_score': situation['economic_score'],
            'resilience_score': situation['resilience_score'], 'adoption_potential': situation['adoption_potential'],
            'specific_needs': situation['specific_needs'], 'strengths': situation['strengths'],
            'vulnerabilities': situation['vulnerabilities'], 'current_caet': getattr(agent, 'caet_tot', 0),
            'adopts_agroecology': agent.adopts_agroecology, 'farm_area': getattr(agent, 'farm_area', 0),
            'family_composition': getattr(agent, 'family_composition', {})
        }

    def get_zone_situation_report(self) -> Dict[str, Any]:
        if not hasattr(self, 'zone_situation_profile'): return {"error": "Analyse non disponible"}
        profile = self.zone_situation_profile
        total_agents = len(self.all_agents)
        recommendations = self._generate_strategic_recommendations()
        return {
            'zone': self.zone_name, 'scenario': self.scenario_type, 'total_agents': total_agents,
            'situation_profile': profile, 'category_distribution': {'vulnerable': profile['vulnerable_agents'], 'emerging': profile['emerging_agents'], 'consolidated': profile['consolidated_agents'], 'advanced': profile['advanced_agents']},
            'key_indicators': {'avg_technology_level': profile['avg_technology_level'], 'avg_family_labor': profile['avg_family_labor'], 'adoption_potential': profile['adoption_potential']},
            'strategic_recommendations': recommendations, 'scenario_compatibility': self._assess_scenario_compatibility()
        }

    def _generate_strategic_recommendations(self) -> List[str]:
        profile = self.zone_situation_profile
        total_agents = len(self.all_agents)
        recommendations = []
        vulnerable_ratio = profile['vulnerable_agents'] / total_agents
        advanced_ratio = profile['advanced_agents'] / total_agents
        
        if vulnerable_ratio > 0.4: recommendations.extend(["Programme de soutien d'urgence pour les exploitations vulnérables", "Formation technique de base prioritaire", "Accès facilité aux intrants essentiels", "Système de microcrédit adapté"])
        if advanced_ratio > 0.3: recommendations.extend(["Politique d'innovation et de démonstration", "Accès aux marchés spécialisés et à l'export", "Programme de mentorat par les exploitants avancés", "Participation à la conception des politiques"])
        if profile['avg_technology_level'] < 40: recommendations.append("Plan de modernisation technologique progressive")
        if profile['adoption_potential'] < 0.4: recommendations.append("Campagne de sensibilisation et démonstration")
        return recommendations

    def _assess_scenario_compatibility(self) -> Dict[str, Any]:
        profile = self.zone_situation_profile
        scenario = self.scenario_prospectif
        compatibility_score = 0.0
        strengths = []
        challenges = []
        
        if scenario.type_scenario == TypeScenario.DESIRABLE:
            if profile['advanced_agents'] > len(self.all_agents) * 0.2: compatibility_score += 0.3; strengths.append("Base d'exploitants avancés solide")
            else: compatibility_score -= 0.2; challenges.append("Base d'exploitants avancés insuffisante")
        if scenario.type_scenario == TypeScenario.CRITIQUE:
            if profile['vulnerable_agents'] > len(self.all_agents) * 0.3: compatibility_score -= 0.4; challenges.append("Forte proportion d'exploitations vulnérables")
            else: compatibility_score += 0.2; strengths.append("Résilience face aux scénarios difficiles")
        if profile['avg_technology_level'] > 50: compatibility_score += 0.2; strengths.append("Niveau technologique adapté")
        else: compatibility_score -= 0.1; challenges.append("Niveau technologique limitant")
        
        return {'compatibility_score': max(0, min(1, 0.5 + compatibility_score)), 'strengths': strengths, 'challenges': challenges, 'recommendation': "Favorable" if compatibility_score > 0 else "À reconsidérer"}

    def get_prospective_analysis(self) -> Dict[str, Any]:
        try:
            scenario = self.scenario_prospectif
            return {
                'scenario_name': scenario.nom, 'scenario_type': scenario.type_scenario.value, 'scenario_code': scenario.code,
                'description': scenario.description, 'implementation_score': self._calculate_agroecology_implementation(1, 1),
                'coherence_score': self._calculate_territorial_coherence(1, 1), 'viability_score': self._calculate_scenario_viability(1, 1),
                'leviers': scenario.leviers, 'contraintes': scenario.contraintes, 'points_basculement': scenario.points_basculement,
                'configuration_forces': scenario.configuration, 'zone': self.zone_name, 'total_agents': len(self.all_agents)
            }
        except Exception as e:
            print(f"❌ Erreur analyse prospective: {e}")
            return {}

    def get_model_statistics(self) -> Dict[str, Any]:
        try:
            if not hasattr(self, 'all_agents') or not self.all_agents: return {}
            caet_scores = [agent.caet_tot for agent in self.all_agents if hasattr(agent, 'caet_tot')]
            adoption_rates = [agent.adopts_agroecology for agent in self.all_agents]
            initial_caet_scores = [getattr(agent, 'initial_caet_tot', agent.caet_tot) for agent in self.all_agents if hasattr(agent, 'caet_tot')]
            avg_caet = np.mean(caet_scores) if caet_scores else 0
            avg_initial_caet = np.mean(initial_caet_scores) if initial_caet_scores else 0
            return {
                'total_agents': len(self.all_agents), 'avg_caet': avg_caet, 'avg_initial_caet': avg_initial_caet,
                'caet_evolution': avg_caet - avg_initial_caet, 'adoption_rate': np.mean(adoption_rates) if adoption_rates else 0,
                'success_rate': self.successful_steps / self.total_steps if self.total_steps > 0 else 0,
                'current_year': self.current_year, 'scenario_type': self.scenario_type, 'zone': self.zone_name
            }
        except Exception as e:
            print(f"❌ Erreur statistiques modèle: {e}")
            return {}

    def __repr__(self) -> str:
        stats = self.get_model_statistics()
        return (f"ZoneAgroecologicalModel(zone={self.zone_name}, scenario={self.scenario_type}, agents={stats.get('total_agents', 0)}, CAET={stats.get('avg_caet', 0):.1f}, adoption={stats.get('adoption_rate', 0)*100:.1f}%)")

# === 5. FONCTIONS POUR CRÉER DES DONNÉES SIMULÉES ===

def create_complete_agent_data(num_agents: int = 285) -> pd.DataFrame:
    """Crée un jeu de données d'agents complet et réaliste pour la simulation"""
    print(f"🧪 Création de {num_agents} agents simulés...")
    np.random.seed(42)
    agents_data = []
    
    for i in range(num_agents):
        zone = "OPIB" if np.random.random() < 0.6 else "OHVN"
        if zone == "OPIB":
            base_caet = np.random.normal(45, 12)
            base_revenue = np.random.normal(1800000, 500000)
        else:
            base_caet = np.random.normal(40, 15) 
            base_revenue = np.random.normal(1500000, 600000)
        
        agent = {
            'unique_id': i, 'zone': zone, 'caet_tot': max(10, min(100, base_caet)), 'farm_revenue': max(100000, base_revenue),
            'adopts_agroecology': np.random.random() < 0.3, 'farm_area': np.random.uniform(1, 50), 'soil_health': np.random.normal(55, 15),
            'value_added': np.random.normal(35, 10), 'profit_margin': np.random.normal(25, 8), 'market_access': np.random.normal(60, 20),
            'biodiversity_index': np.random.normal(35, 12), 'water_management': np.random.normal(50, 15), 'social_cohesion': np.random.normal(65, 15),
            'knowledge_sharing': np.random.normal(55, 15), 'community_participation': np.random.normal(60, 15),
        }
        
        caet_dimensions = {'div_score': 0.14, 'syn_score': 0.12, 'eff_score': 0.09, 'rec_score': 0.08, 'res_score': 0.16, 'cultf_score': 0.06, 'cocr_score': 0.09, 'human_score': 0.11, 'circ_score': 0.08, 'respg_score': 0.07}
        for dim, weight in caet_dimensions.items():
            agent[dim] = max(0, min(100, agent['caet_tot'] * weight * np.random.uniform(0.8, 1.2)))
        
        technologies = ["crops", "animals", "trees", "animnum", "cla_int", "s_plant", "tree_int", "connectivity", "ext_inp", "soil_fert", "pest_dis", "vuln", "rec_biomass", "water", "seeds_breeds", "ren_energy", "waste", "indebt", "diet", "food-heritage", "food-self-suff", "platforms", "ae_know", "partic_orgs", "women", "labour", "youth", "animalwel", "mkt_local", "networks", "local_fs", "prod_empow", "prod_orgs", "partic_prod", "coalanwel"]
        for tech in technologies:
            agent[tech] = np.random.uniform(0, 4)
        
        family_composition = {
            'hh_men': max(1, int(np.random.poisson(1.5))), 'hh_women': max(1, int(np.random.poisson(1.3))),
            'hh_myoung': int(np.random.poisson(0.8)), 'hh_fyoung': int(np.random.poisson(0.7)),
            'hh_children': int(np.random.poisson(1.2)), 'hh_fem': int(np.random.poisson(1.0))
        }
        agent.update(family_composition)
        agents_data.append(agent)
    
    print(f"✅ {len(agents_data)} agents créés avec succès")
    return pd.DataFrame(agents_data)


In [3]:
# === IMPORTS GLOBAUX ===
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# === SIMULATION 2025-2045 AVEC TOUS LES SCÉNARIOS - VERSION CORRIGÉE ===

def run_complete_simulation_all_scenarios(file_path: str, export_path: str = "simulation_complete_2025_2045"):
    """
    Simulation complète 2025-2045 avec TOUS les scénarios pour les deux zones
    Utilise les données réelles exactes sans créer d'agents supplémentaires
    """
    
    # Créer le dossier d'export
    if not os.path.exists(export_path):
        os.makedirs(export_path)
    
    print("🚀 LANCEMENT DE LA SIMULATION COMPLÈTE 2025-2045")
    print("=" * 60)
    
    # 1. Chargement des données réelles
    print("📂 CHARGEMENT DES DONNÉES...")
    real_data = load_and_clean_data(file_path)
    
    if real_data.empty:
        print("❌ Impossible de charger les données")
        return None
    
    # 2. Analyse de la répartition naturelle des agents
    print("\n📊 ANALYSE DE LA RÉPARTITION NATURELLE DES AGENTS:")
    
    # Vérifier et préparer la colonne zone
    if 'zone' not in real_data.columns:
        print("🔍 Détermination automatique des zones...")
        real_data = determine_zones_from_data(real_data)
    
    zone_counts = real_data['zone'].value_counts()
    opib_count = zone_counts.get('OPIB', 0)
    ohvn_count = zone_counts.get('OHVN', 0)
    total_agents = len(real_data)
    
    print(f"   • OPIB: {opib_count} agents")
    print(f"   • OHVN: {ohvn_count} agents")
    print(f"   • Total: {total_agents} agents")
    print(f"   • Utilisation de la répartition naturelle des données")
    
    # 3. TOUS LES SCÉNARIOS À SIMULER
    print("\n🎯 CONFIGURATION DES SCÉNARIOS:")
    
    # Tous les scénarios OPIB
    scenarios_opib = [
        "actuel", 
        "baguineda_gnogon_te", 
        "baguineda_en_mouvement", 
        "baguineda_to_be", 
        "baguineda_pole_urbain", 
        "baguineda_baa_kedi"
    ]
    
    # Tous les scénarios OHVN
    scenarios_ohvn = [
        "actuel",
        "ohvn_pivot_agroindustriel", 
        "ohvn_ecosysteme_resilient",
        "ohvn_agriculture_precision", 
        "ohvn_corridor_economique", 
        "ohvn_territoires_fragmentes"
    ]
    
    print(f"   • Scénarios OPIB: {len(scenarios_opib)}")
    print(f"   • Scénarios OHVN: {len(scenarios_ohvn)}")
    print(f"   • Total scénarios: {len(scenarios_opib) + len(scenarios_ohvn)}")
    
    # 4. Paramètres de simulation
    start_year = 2025
    end_year = 2045
    simulation_years = end_year - start_year
    
    print(f"\n⏰ PÉRIODE DE SIMULATION: {start_year}-{end_year} ({simulation_years} années)")
    
    # Stockage des résultats
    all_results = {
        'model': [],
        'agents': [],
        'prospective': [],
        'analysis': []
    }
    
    # 5. SIMULATION COMPLÈTE ZONE OPIB - TOUS LES SCÉNARIOS
    print(f"\n🌾 SIMULATION ZONE OPIB - {opib_count} AGENTS - {len(scenarios_opib)} SCÉNARIOS")
    print("-" * 50)
    
    opib_data = real_data[real_data['zone'] == 'OPIB']
    
    if len(opib_data) > 0:
        for i, scenario in enumerate(scenarios_opib, 1):
            print(f"   [{i}/{len(scenarios_opib)}] 🎯 Scénario: {scenario}")
            
            try:
                model = ZoneAgroecologicalModel(
                    zone_name="OPIB",
                    agent_data=opib_data,
                    scenario_type=scenario,
                    num_agents=len(opib_data),
                    start_year=start_year
                )
                
                # Exécuter la simulation
                model_results, agent_results, prospective_results = model.run_model(steps=simulation_years)
                
                # Ajouter métadonnées
                model_results['scenario_type'] = scenario
                model_results['zone_type'] = 'OPIB'
                agent_results['scenario_type'] = scenario
                agent_results['zone_type'] = 'OPIB'
                prospective_results['scenario_type'] = scenario
                prospective_results['zone_type'] = 'OPIB'
                
                # Stocker les résultats
                all_results['model'].append(model_results)
                all_results['agents'].append(agent_results)
                all_results['prospective'].append(prospective_results)
                
                # Analyse détaillée
                analysis_data = {
                    'zone': 'OPIB',
                    'scenario': scenario,
                    'agent_count': len(opib_data),
                    'model_statistics': model.get_model_statistics(),
                    'zone_situation_report': model.get_zone_situation_report(),
                    'prospective_analysis': model.get_prospective_analysis(),
                    'simulation_period': f"{start_year}-{end_year}",
                    'timestamp': datetime.now().isoformat()
                }
                all_results['analysis'].append(analysis_data)
                
                print(f"      ✅ {scenario} terminé - {len(agent_results)} enregistrements")
                
            except Exception as e:
                print(f"      ❌ Erreur {scenario}: {e}")
                continue
    else:
        print("   ⚠️ Aucun agent OPIB trouvé dans les données")
    
    # 6. SIMULATION COMPLÈTE ZONE OHVN - TOUS LES SCÉNARIOS
    print(f"\n🌳 SIMULATION ZONE OHVN - {ohvn_count} AGENTS - {len(scenarios_ohvn)} SCÉNARIOS")
    print("-" * 50)
    
    ohvn_data = real_data[real_data['zone'] == 'OHVN']
    
    if len(ohvn_data) > 0:
        for i, scenario in enumerate(scenarios_ohvn, 1):
            print(f"   [{i}/{len(scenarios_ohvn)}] 🎯 Scénario: {scenario}")
            
            try:
                model = ZoneAgroecologicalModel(
                    zone_name="OHVN",
                    agent_data=ohvn_data,
                    scenario_type=scenario,
                    num_agents=len(ohvn_data),
                    start_year=start_year
                )
                
                # Exécuter la simulation
                model_results, agent_results, prospective_results = model.run_model(steps=simulation_years)
                
                # Ajouter métadonnées
                model_results['scenario_type'] = scenario
                model_results['zone_type'] = 'OHVN'
                agent_results['scenario_type'] = scenario
                agent_results['zone_type'] = 'OHVN'
                prospective_results['scenario_type'] = scenario
                prospective_results['zone_type'] = 'OHVN'
                
                # Stocker les résultats
                all_results['model'].append(model_results)
                all_results['agents'].append(agent_results)
                all_results['prospective'].append(prospective_results)
                
                # Analyse détaillée
                analysis_data = {
                    'zone': 'OHVN',
                    'scenario': scenario,
                    'agent_count': len(ohvn_data),
                    'model_statistics': model.get_model_statistics(),
                    'zone_situation_report': model.get_zone_situation_report(),
                    'prospective_analysis': model.get_prospective_analysis(),
                    'simulation_period': f"{start_year}-{end_year}",
                    'timestamp': datetime.now().isoformat()
                }
                all_results['analysis'].append(analysis_data)
                
                print(f"      ✅ {scenario} terminé - {len(agent_results)} enregistrements")
                
            except Exception as e:
                print(f"      ❌ Erreur {scenario}: {e}")
                continue
    else:
        print("   ⚠️ Aucun agent OHVN trouvé dans les données")
    
    # 7. EXPORT COMPLET DES RÉSULTATS EN EXCEL
    print("\n💾 EXPORT COMPLET DES RÉSULTATS EN EXCEL...")
    
    try:
        # Vérifier s'il y a des résultats
        if not all_results['model']:
            print("❌ Aucun résultat à exporter")
            return None
            
        # Combiner tous les résultats
        final_model = pd.concat(all_results['model'], ignore_index=True)
        final_agents = pd.concat(all_results['agents'], ignore_index=True)
        final_prospective = pd.concat(all_results['prospective'], ignore_index=True)
        
        print(f"   • Données modèles: {len(final_model)} lignes")
        print(f"   • Données agents: {len(final_agents)} lignes")
        print(f"   • Données prospectives: {len(final_prospective)} lignes")
        
        # Créer le fichier Excel complet avec multiples onglets
        excel_file = os.path.join(export_path, "simulation_complete_2025_2045.xlsx")
        
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            # === ONGLET 1: Résultats principaux des modèles ===
            final_model.to_excel(writer, sheet_name='Resultats_Modeles', index=False)
            print("      ✅ Onglet 'Resultats_Modeles' créé")
            
            # === ONGLET 2: Données détaillées des agents ===
            final_agents.to_excel(writer, sheet_name='Donnees_Agents_Details', index=False)
            print("      ✅ Onglet 'Donnees_Agents_Details' créé")
            
            # === ONGLET 3: Analyse prospective ===
            final_prospective.to_excel(writer, sheet_name='Analyse_Prospective', index=False)
            print("      ✅ Onglet 'Analyse_Prospective' créé")
            
            # === ONGLET 4: Statistiques synthèses par scénario ===
            stats_summary = create_complete_statistics_summary(final_model)
            stats_summary.to_excel(writer, sheet_name='Statistiques_Synthese', index=False)
            print("      ✅ Onglet 'Statistiques_Synthese' créé")
            
            # === ONGLET 5: Évolution temporelle détaillée ===
            evolution_data = create_detailed_evolution_analysis(final_model)
            evolution_data.to_excel(writer, sheet_name='Evolution_Temporelle', index=False)
            print("      ✅ Onglet 'Evolution_Temporelle' créé")
            
            # === ONGLET 6: Comparaison complète des scénarios ===
            scenario_comparison = create_comprehensive_scenario_comparison(final_model)
            scenario_comparison.to_excel(writer, sheet_name='Comparaison_Scenarios', index=False)
            print("      ✅ Onglet 'Comparaison_Scenarios' créé")
            
            # === ONGLET 7: Métadonnées de simulation ===
            metadata = create_detailed_metadata(real_data, all_results['analysis'], scenarios_opib + scenarios_ohvn)
            metadata.to_excel(writer, sheet_name='Metadonnees_Simulation', index=False)
            print("      ✅ Onglet 'Metadonnees_Simulation' créé")
            
            # === ONGLET 8: Analyse des performances par scénario ===
            performance_analysis = create_performance_analysis(final_model)
            performance_analysis.to_excel(writer, sheet_name='Analyse_Performances', index=False)
            print("      ✅ Onglet 'Analyse_Performances' créé")
        
        print(f"✅ Fichier Excel complet créé: {excel_file}")
        
        # 8. GÉNÉRATION DE GRAPHIQUES COMPLETS
        print("\n📈 GÉNÉRATION DES GRAPHIQUES COMPLETS...")
        generate_comprehensive_charts(final_model, export_path)
        
        # 9. RAPPORT FINAL DÉTAILLÉ
        print("\n📋 GÉNÉRATION DU RAPPORT FINAL DÉTAILLÉ...")
        create_detailed_final_report(final_model, all_results['analysis'], export_path, start_year, end_year)
        
        # 10. SYNTHÈSE DES RÉSULTATS
        print("\n🔍 CRÉATION DE LA SYNTHÈSE DES RÉSULTATS...")
        create_results_synthesis(final_model, all_results['analysis'], export_path)
        
        print(f"\n🎉 SIMULATION COMPLÈTE TERMINÉE AVEC SUCCÈS!")
        print("=" * 60)
        print(f"📁 Dossier résultats: {export_path}")
        print(f"📊 Fichier Excel: {excel_file}")
        print(f"👥 Agents réels utilisés: {total_agents}")
        print(f"🌾 OPIB: {opib_count} agents, {len(scenarios_opib)} scénarios")
        print(f"🌳 OHVN: {ohvn_count} agents, {len(scenarios_ohvn)} scénarios")
        print(f"🎯 Total scénarios simulés: {len(scenarios_opib) + len(scenarios_ohvn)}")
        print(f"📈 Période: {start_year}-{end_year} ({simulation_years} années)")
        
        return {
            'model_data': final_model,
            'agent_data': final_agents,
            'prospective_data': final_prospective,
            'analysis_data': all_results['analysis'],
            'excel_file': excel_file,
            'original_data': real_data,
            'scenarios_simulated': {
                'OPIB': scenarios_opib,
                'OHVN': scenarios_ohvn
            }
        }
        
    except Exception as e:
        print(f"❌ Erreur lors de l'export: {e}")
        import traceback
        traceback.print_exc()
        return None

def determine_zones_from_data(data: pd.DataFrame) -> pd.DataFrame:
    """Détermine automatiquement les zones à partir des données disponibles"""
    
    if 'Village' in data.columns:
        print("   🔍 Utilisation de la colonne 'Village' pour déterminer les zones")
        data['zone'] = data['Village'].apply(
            lambda x: 'OPIB' if 'baguineda' in str(x).lower() else 
                     'OHVN' if 'tiele' in str(x).lower() else 
                     'OPIB'  # Par défaut OPIB
        )
    elif 'village' in data.columns:
        print("   🔍 Utilisation de la colonne 'village' pour déterminer les zones")
        data['zone'] = data['village'].apply(
            lambda x: 'OPIB' if 'baguineda' in str(x).lower() else 
                     'OHVN' if 'tiele' in str(x).lower() else 
                     'OPIB'
        )
    else:
        print("   ⚠️ Aucune colonne de localisation trouvée, attribution aléatoire")
        # Attribution basée sur l'ID pour avoir une répartition équilibrée
        data['zone'] = ['OPIB' if i % 2 == 0 else 'OHVN' for i in range(len(data))]
    
    return data

def create_complete_statistics_summary(model_data: pd.DataFrame) -> pd.DataFrame:
    """Crée un tableau complet de statistiques synthèses"""
    summary = []
    
    # Années clés pour l'analyse
    key_years = [2025, 2030, 2035, 2040, 2045]
    
    for zone in model_data['zone_type'].unique():
        zone_data = model_data[model_data['zone_type'] == zone]
        for scenario in zone_data['scenario_type'].unique():
            scenario_data = zone_data[zone_data['scenario_type'] == scenario]
            
            for year in key_years:
                year_data = scenario_data[scenario_data['year'] == year]
                
                if len(year_data) > 0:
                    stats = {
                        'Zone': zone,
                        'Scénario': scenario,
                        'Année': year,
                        'Nb_Agents': len(year_data['agent_id'].unique()),
                        'CAET_Moyen': round(year_data['performance_globale'].mean() * 100, 2),
                        'CAET_Min': round(year_data['performance_globale'].min() * 100, 2),
                        'CAET_Max': round(year_data['performance_globale'].max() * 100, 2),
                        'Taux_Adoption_Agroeco': round(year_data['adopts_agroecology'].mean() * 100, 2),
                        'Score_Resilience': round(year_data.get('resilience', 0).mean() * 100, 2) if 'resilience' in year_data.columns else 0,
                    }
                    
                    # Ajouter tous les indicateurs de performance disponibles
                    performance_indicators = ['soil_health', 'biodiversity_index', 'farm_revenue', 
                                            'value_added', 'profit_margin', 'market_access']
                    
                    for indicator in performance_indicators:
                        if indicator in year_data.columns:
                            stats[indicator] = round(year_data[indicator].mean(), 2)
                    
                    summary.append(stats)
    
    return pd.DataFrame(summary)

def create_detailed_evolution_analysis(model_data: pd.DataFrame) -> pd.DataFrame:
    """Analyse détaillée de l'évolution temporelle"""
    evolution = []
    
    # Toutes les années de simulation
    all_years = sorted(model_data['year'].unique())
    
    for zone in model_data['zone_type'].unique():
        zone_data = model_data[model_data['zone_type'] == zone]
        for scenario in zone_data['scenario_type'].unique():
            scenario_data = zone_data[zone_data['scenario_type'] == scenario]
            
            for year in all_years:
                year_data = scenario_data[scenario_data['year'] == year]
                
                if len(year_data) > 0:
                    evolution.append({
                        'Zone': zone,
                        'Scénario': scenario,
                        'Année': year,
                        'Nb_Agents': len(year_data['agent_id'].unique()),
                        'CAET_Moyen': round(year_data['performance_globale'].mean() * 100, 2),
                        'Taux_Adoption': round(year_data['adopts_agroecology'].mean() * 100, 2),
                        'Evolution_CAET_vs_2025': round(
                            (year_data['performance_globale'].mean() - 
                             model_data[(model_data['zone_type'] == zone) & 
                                      (model_data['scenario_type'] == scenario) & 
                                      (model_data['year'] == 2025)]['performance_globale'].mean()) * 100, 2
                        ) if 2025 in all_years else 0
                    })
    
    return pd.DataFrame(evolution)

def create_comprehensive_scenario_comparison(model_data: pd.DataFrame) -> pd.DataFrame:
    """Comparaison complète entre tous les scénarios"""
    comparison = []
    
    # Données finales (2045)
    final_data = model_data[model_data['year'] == 2045]
    
    for zone in final_data['zone_type'].unique():
        zone_data = final_data[final_data['zone_type'] == zone]
        
        for scenario in zone_data['scenario_type'].unique():
            scenario_data = zone_data[zone_data['scenario_type'] == scenario]
            
            if len(scenario_data) > 0:
                comp = {
                    'Zone': zone,
                    'Scénario': scenario,
                    'Classement_CAET': 0,  # Sera calculé après
                    'CAET_Final': round(scenario_data['performance_globale'].mean() * 100, 2),
                    'Taux_Adoption_Final': round(scenario_data['adopts_agroecology'].mean() * 100, 2),
                    'Gain_CAET_vs_2025': round(
                        (scenario_data['performance_globale'].mean() - 
                         model_data[(model_data['zone_type'] == zone) & 
                                  (model_data['scenario_type'] == scenario) & 
                                  (model_data['year'] == 2025)]['performance_globale'].mean()) * 100, 2
                    ),
                    'Nb_Agents': len(scenario_data['agent_id'].unique())
                }
                
                # Ajouter indicateurs de performance finaux
                for indicator in ['soil_health', 'biodiversity_index', 'farm_revenue']:
                    if indicator in scenario_data.columns:
                        comp[f'{indicator}_final'] = round(scenario_data[indicator].mean(), 2)
                
                comparison.append(comp)
        
        # Calcul du classement par zone
        zone_comparisons = [c for c in comparison if c['Zone'] == zone]
        zone_comparisons_sorted = sorted(zone_comparisons, key=lambda x: x['CAET_Final'], reverse=True)
        
        for i, comp in enumerate(zone_comparisons_sorted, 1):
            comp['Classement_CAET'] = i
    
    return pd.DataFrame(comparison)

def create_detailed_metadata(original_data: pd.DataFrame, analysis_data: list, all_scenarios: list) -> pd.DataFrame:
    """Crée un tableau détaillé de métadonnées"""
    
    metadata = []
    
    # Informations générales
    metadata.append({'Catégorie': 'GÉNÉRAL', 'Variable': 'Date Simulation', 'Valeur': datetime.now().strftime('%Y-%m-%d %H:%M:%S')})
    metadata.append({'Catégorie': 'GÉNÉRAL', 'Variable': 'Période', 'Valeur': '2025-2045'})
    metadata.append({'Catégorie': 'GÉNÉRAL', 'Variable': 'Années Simulées', 'Valeur': '21'})
    
    # Données sources
    metadata.append({'Catégorie': 'DONNÉES SOURCES', 'Variable': 'Total Agents', 'Valeur': len(original_data)})
    metadata.append({'Catégorie': 'DONNÉES SOURCES', 'Variable': 'Agents OPIB', 'Valeur': len(original_data[original_data['zone'] == 'OPIB'])})
    metadata.append({'Catégorie': 'DONNÉES SOURCES', 'Variable': 'Agents OHVN', 'Valeur': len(original_data[original_data['zone'] == 'OHVN'])})
    
    # Scénarios simulés
    metadata.append({'Catégorie': 'SCÉNARIOS', 'Variable': 'Total Scénarios', 'Valeur': len(all_scenarios)})
    metadata.append({'Catégorie': 'SCÉNARIOS', 'Variable': 'Scénarios OPIB', 'Valeur': len([s for s in all_scenarios if 'baguineda' in s])})
    metadata.append({'Catégorie': 'SCÉNARIOS', 'Variable': 'Scénarios OHVN', 'Valeur': len([s for s in all_scenarios if 'ohvn' in s])})
    
    # Résultats de simulation
    if analysis_data:
        successful_simulations = len(analysis_data)
        metadata.append({'Catégorie': 'RÉSULTATS', 'Variable': 'Simulations Réussies', 'Valeur': successful_simulations})
        metadata.append({'Catégorie': 'RÉSULTATS', 'Variable': 'Taux Réussite', 'Valeur': f"{(successful_simulations/len(all_scenarios))*100:.1f}%"})
    
    return pd.DataFrame(metadata)

def create_performance_analysis(model_data: pd.DataFrame) -> pd.DataFrame:
    """Analyse détaillée des performances par scénario"""
    performance = []
    
    final_data = model_data[model_data['year'] == 2045]
    
    for zone in final_data['zone_type'].unique():
        zone_data = final_data[final_data['zone_type'] == zone]
        
        for scenario in zone_data['scenario_type'].unique():
            scenario_data = zone_data[zone_data['scenario_type'] == scenario]
            
            if len(scenario_data) > 0:
                perf = {
                    'Zone': zone,
                    'Scénario': scenario,
                    'Performance_Globale': round(scenario_data['performance_globale'].mean() * 100, 2),
                    'Resilience_Ecologique': round(scenario_data.get('ecological_resilience', 0).mean(), 2) if 'ecological_resilience' in scenario_data.columns else 0,
                    'Sante_Sols': round(scenario_data.get('soil_health', 0).mean(), 2) if 'soil_health' in scenario_data.columns else 0,
                    'Biodiversite': round(scenario_data.get('biodiversity_index', 0).mean(), 2) if 'biodiversity_index' in scenario_data.columns else 0,
                    'Revenu_Agricole': round(scenario_data.get('farm_revenue', 0).mean(), 2) if 'farm_revenue' in scenario_data.columns else 0,
                    'Adoption_Agroeco': round(scenario_data['adopts_agroecology'].mean() * 100, 2),
                    'Score_Durabilite': 0  # Calculé après
                }
                
                # Calcul du score de durabilité (moyenne pondérée)
                sustainability_indicators = [
                    perf['Performance_Globale'] * 0.3,
                    perf['Resilience_Ecologique'] * 0.2,
                    perf['Sante_Sols'] * 0.2,
                    perf['Biodiversite'] * 0.15,
                    perf['Adoption_Agroeco'] * 0.15
                ]
                perf['Score_Durabilite'] = round(sum(sustainability_indicators), 2)
                
                performance.append(perf)
    
    return pd.DataFrame(performance)

def generate_comprehensive_charts(model_data: pd.DataFrame, export_path: str):
    """Génère des graphiques complets pour l'analyse"""
    try:
        plt.style.use('default')
        sns.set_palette("husl")
        
        # 1. Évolution du CAET pour tous les scénarios
        plt.figure(figsize=(15, 8))
        
        for zone in model_data['zone_type'].unique():
            zone_data = model_data[model_data['zone_type'] == zone]
            for scenario in zone_data['scenario_type'].unique():
                scenario_data = zone_data[zone_data['scenario_type'] == scenario]
                yearly_avg = scenario_data.groupby('year')['performance_globale'].mean().reset_index()
                
                plt.plot(yearly_avg['year'], yearly_avg['performance_globale'] * 100, 
                        label=f'{zone} - {scenario}', linewidth=2, marker='o', markersize=3)
        
        plt.title('Évolution du Score CAET - Tous les Scénarios (2025-2045)', fontsize=14)
        plt.xlabel('Année')
        plt.ylabel('Score CAET (%)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(export_path, 'evolution_caet_tous_scenarios.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        print("      ✅ Graphique évolution CAET généré")
        
    except Exception as e:
        print(f"      ⚠️ Erreur génération graphiques: {e}")

def create_detailed_final_report(model_data: pd.DataFrame, analysis_data: list, export_path: str, start_year: int, end_year: int):
    """Crée un rapport final détaillé"""
    
    report_file = os.path.join(export_path, "rapport_final_detaille.txt")
    
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("RAPPORT FINAL DÉTAILLÉ - SIMULATION AGROÉCOLOGIQUE 2025-2045\n")
        f.write("=" * 70 + "\n\n")
        
        f.write(f"Période de simulation: {start_year}-{end_year}\n")
        f.write(f"Date de génération: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        # Analyse par zone
        final_data = model_data[model_data['year'] == end_year]
        
        for zone in final_data['zone_type'].unique():
            f.write(f"\n{zone} - ANALYSE DES SCÉNARIOS:\n")
            f.write("-" * 50 + "\n")
            
            zone_data = final_data[final_data['zone_type'] == zone]
            scenarios_sorted = zone_data.groupby('scenario_type')['performance_globale'].mean().sort_values(ascending=False)
            
            for i, (scenario, score) in enumerate(scenarios_sorted.items(), 1):
                scenario_data = zone_data[zone_data['scenario_type'] == scenario]
                adoption_rate = scenario_data['adopts_agroecology'].mean() * 100
                
                f.write(f"\n{i}. {scenario}:\n")
                f.write(f"   • Score CAET: {score*100:.1f}%\n")
                f.write(f"   • Taux d'adoption: {adoption_rate:.1f}%\n")
                f.write(f"   • Nombre d'agents: {len(scenario_data['agent_id'].unique())}\n")
        
        f.write("\n" + "=" * 70 + "\n")
        f.write("RECOMMANDATIONS STRATÉGIQUES:\n")
        f.write("-" * 50 + "\n")
        
        # Recommandations basées sur les résultats
        for zone in final_data['zone_type'].unique():
            zone_data = final_data[final_data['zone_type'] == zone]
            best_scenario = zone_data.groupby('scenario_type')['performance_globale'].mean().idxmax()
            
            f.write(f"\n{zone}:\n")
            f.write(f"• Scénario recommandé: {best_scenario}\n")
            f.write(f"• Focus: Investir dans les politiques soutenant ce scénario\n")
        
        f.write("\n" + "=" * 70 + "\n")
        f.write("FIN DU RAPPORT\n")
    
    print("      ✅ Rapport final généré")

def create_results_synthesis(model_data: pd.DataFrame, analysis_data: list, export_path: str):
    """Crée une synthèse des résultats"""
    synthesis_file = os.path.join(export_path, "synthese_resultats.txt")
    
    with open(synthesis_file, 'w', encoding='utf-8') as f:
        f.write("SYNTHÈSE DES RÉSULTATS - SIMULATION 2025-2045\n")
        f.write("=" * 50 + "\n\n")
        
        final_data = model_data[model_data['year'] == 2045]
        
        f.write("MEILLEURS SCÉNARIOS PAR ZONE:\n")
        f.write("-" * 30 + "\n")
        
        for zone in final_data['zone_type'].unique():
            zone_data = final_data[final_data['zone_type'] == zone]
            best_scenario = zone_data.groupby('scenario_type')['performance_globale'].mean().idxmax()
            best_score = zone_data.groupby('scenario_type')['performance_globale'].mean().max() * 100
            
            f.write(f"\n{zone}: {best_scenario} ({best_score:.1f}% CAET)\n")
        
        f.write("\n" + "=" * 50 + "\n")
        f.write("© Simulation Agroécologique 2025-2045\n")
    
    print("      ✅ Synthèse des résultats générée")

# === LANCEMENT DE LA SIMULATION COMPLÈTE ===
if __name__ == "__main__":
    """
    Point d'entrée principal - Lance la simulation complète avec tous les scénarios
    """
    print("🌍 DÉMARRAGE DE LA SIMULATION COMPLÈTE AVEC TOUS LES SCÉNARIOS")
    
    # Chemin vers votre fichier de données
    data_file = r"C:\Users\dell\Desktop\TAPE_final.xlsx"
    
    # Lancer la simulation complète
    results = run_complete_simulation_all_scenarios(
        file_path=data_file,
        export_path="simulation_complete_results"
    )
    
    if results:
        print("\n✅ SIMULATION COMPLÈTE RÉUSSIE!")
        print(f"📊 Données originales utilisées: {len(results['original_data'])} agents")
        print(f"🎯 Scénarios simulés: {len(results['scenarios_simulated']['OPIB']) + len(results['scenarios_simulated']['OHVN'])}")
        print(f"💾 Fichier résultats: {results['excel_file']}")
    else:
        print("\n❌ La simulation complète a échoué")

🌍 DÉMARRAGE DE LA SIMULATION COMPLÈTE AVEC TOUS LES SCÉNARIOS
🚀 LANCEMENT DE LA SIMULATION COMPLÈTE 2025-2045
📂 CHARGEMENT DES DONNÉES...

📊 ANALYSE DE LA RÉPARTITION NATURELLE DES AGENTS:
   • OPIB: 143 agents
   • OHVN: 142 agents
   • Total: 285 agents
   • Utilisation de la répartition naturelle des données

🎯 CONFIGURATION DES SCÉNARIOS:
   • Scénarios OPIB: 6
   • Scénarios OHVN: 6
   • Total scénarios: 12

⏰ PÉRIODE DE SIMULATION: 2025-2045 (20 années)

🌾 SIMULATION ZONE OPIB - 143 AGENTS - 6 SCÉNARIOS
--------------------------------------------------
   [1/6] 🎯 Scénario: actuel
✅ 5 scénarios OPIB chargés
✅ Modèle OPIB initialisé - 143 agents - Scénario: actuel
   🎮 Simulation OPIB - actuel (20 steps)
      ✅ actuel terminé - 3003 enregistrements
   [2/6] 🎯 Scénario: baguineda_gnogon_te
✅ 5 scénarios OPIB chargés
✅ Modèle OPIB initialisé - 143 agents - Scénario: baguineda_gnogon_te
   🎮 Simulation OPIB - baguineda_gnogon_te (20 steps)
      ✅ baguineda_gnogon_te terminé - 3003 

In [4]:
# === IMPORTS GLOBAUX ===
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# === DICTIONNAIRE COMPLET DES INDICATEURS DE PERFORMANCE ===

performance_indicators = {
    "dietary_diversity": "Dietary Diversity", 
    "soil_health": "Soil Health",
    "landtenure": "Land Tenure", 
    "pest_score": "Pesticides",
    "Agricultural_diversity_tape": "Biodiversity", 
    "youth_score": "Youth",
    "tot_productivity": "Productivity", 
    "farm_revenue": "Income",
    "value_added": "Value Added", 
    "wemp_score": "Women",
    "market_access": "Market Access",
    "input_costs": "Input Costs", 
    "profit_margin": "Profit Margin",
    "water_management": "Water Management",
    "carbon_sequestration": "Carbon Sequestration",
    "biodiversity_index": "Biodiversity Index",
    "ecological_resilience": "Ecological Resilience",
    "social_cohesion": "Social Cohesion",
    "knowledge_sharing": "Knowledge Sharing",
    "community_participation": "Community Participation",
    "local_market_integration": "Local Market Integration",
    "value_chain_development": "Value Chain Development",
    "infrastructure_access": "Infrastructure Access",
    "territorial_connectivity": "Territorial Connectivity",
    "circular_economy": "Circular Economy",
    "food_sovereignty": "Food Sovereignty"
}

# === ANALYSE GLOBALE DES RÉSULTATS - VERSION COMPLÈTE CORRIGÉE ===

def analyse_globale_resultats_complete(excel_file: str, export_path: str = "analyse_globale_complete"):
    """
    Analyse globale complète avec tous les indicateurs de performance
    Version corrigée avec imports globaux
    """
    
    if not os.path.exists(export_path):
        os.makedirs(export_path)
    
    print("🌍 ANALYSE GLOBALE COMPLÈTE DES RÉSULTATS")
    print("=" * 60)
    
    # 1. CHARGEMENT DES DONNÉES
    print("📂 CHARGEMENT DES DONNÉES...")
    
    try:
        # Chargement des données principales
        model_data = pd.read_excel(excel_file, sheet_name='Resultats_Modeles')
        agent_data = pd.read_excel(excel_file, sheet_name='Donnees_Agents_Details')
        
        print(f"✅ Données chargées - Modèles: {len(model_data)} lignes, Agents: {len(agent_data)} lignes")
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return None
    
    # 2. IDENTIFICATION DES INDICATEURS DISPONIBLES
    print("\n🔍 IDENTIFICATION DES INDICATEURS DISPONIBLES...")
    
    # Colonnes essentielles
    essential_columns = ['year', 'zone_type', 'scenario_type', 'performance_globale', 'adopts_agroecology']
    
    # Vérifier quelles colonnes de performance sont disponibles
    available_performance_indicators = []
    for indicator in performance_indicators.keys():
        if indicator in model_data.columns:
            available_performance_indicators.append(indicator)
    
    print(f"   ✅ Indicateurs essentiels: {len(essential_columns)}")
    print(f"   ✅ Indicateurs de performance disponibles: {len(available_performance_indicators)}")
    print(f"   📊 Liste: {', '.join(available_performance_indicators[:5])}{'...' if len(available_performance_indicators) > 5 else ''}")
    
    # 3. ANALYSES SPÉCIFIQUES PAR CATÉGORIE D'INDICATEURS
    print("\n📈 ANALYSE PAR CATÉGORIES D'INDICATEURS...")
    
    # Catégorisation des indicateurs
    indicator_categories = {
        'Économique': ['farm_revenue', 'value_added', 'profit_margin', 'market_access', 'input_costs'],
        'Environnemental': ['soil_health', 'biodiversity_index', 'water_management', 'carbon_sequestration', 'ecological_resilience'],
        'Social': ['social_cohesion', 'knowledge_sharing', 'community_participation', 'youth_score', 'wemp_score'],
        'Territorial': ['local_market_integration', 'value_chain_development', 'infrastructure_access', 'territorial_connectivity'],
        'Alimentation': ['dietary_diversity', 'food_sovereignty']
    }
    
    # Identifier les indicateurs disponibles par catégorie
    available_by_category = {}
    for category, indicators in indicator_categories.items():
        available_inds = [ind for ind in indicators if ind in available_performance_indicators]
        if available_inds:
            available_by_category[category] = available_inds
            print(f"   📁 {category}: {len(available_inds)} indicateurs")
    
    # 4. EXÉCUTION DES ANALYSES
    analyse_descriptive_avancee(model_data, available_performance_indicators, available_by_category, export_path)
    analyse_correlation_complete(model_data, available_performance_indicators, export_path)
    analyse_par_categorie(model_data, available_by_category, export_path)
    generer_rapport_complet(model_data, available_performance_indicators, available_by_category, export_path)
    
    print(f"\n✅ ANALYSE GLOBALE COMPLÈTE TERMINÉE!")
    print(f"📁 Dossier résultats: {export_path}")
    
    return {
        'model_data': model_data,
        'available_indicators': available_performance_indicators,
        'available_categories': available_by_category,
        'export_path': export_path
    }

def analyse_descriptive_avancee(model_data: pd.DataFrame, available_indicators: list, available_categories: dict, export_path: str):
    """Analyse descriptive avancée avec tous les indicateurs"""
    
    final_data = model_data[model_data['year'] == 2045]
    
    if final_data.empty:
        print("   ⚠️ Aucune donnée pour 2045")
        return
    
    print("   📊 Analyse descriptive avancée...")
    
    # Créer un rapport descriptif complet
    descriptive_stats = []
    
    for zone in final_data['zone_type'].unique():
        zone_data = final_data[final_data['zone_type'] == zone]
        
        for scenario in zone_data['scenario_type'].unique():
            scenario_data = zone_data[zone_data['scenario_type'] == scenario]
            
            if len(scenario_data) > 0:
                stats = {
                    'Zone': zone,
                    'Scénario': scenario,
                    'Nb_Agents': len(scenario_data),
                    'CAET_Moyen': scenario_data['performance_globale'].mean() * 100,
                    'Taux_Adoption': scenario_data['adopts_agroecology'].mean() * 100
                }
                
                # Ajouter tous les indicateurs disponibles
                for indicator in available_indicators:
                    if indicator in scenario_data.columns:
                        stats[f'{indicator}_moyen'] = scenario_data[indicator].mean()
                        stats[f'{indicator}_std'] = scenario_data[indicator].std()
                
                descriptive_stats.append(stats)
    
    # Sauvegarder les statistiques descriptives
    if descriptive_stats:
        stats_df = pd.DataFrame(descriptive_stats)
        stats_df.to_excel(os.path.join(export_path, 'statistiques_descriptives_completes.xlsx'), index=False)
    
    # Visualisations par catégorie
    for category, indicators in available_categories.items():
        if len(indicators) > 0:
            visualiser_categorie_indicators(final_data, category, indicators, export_path)
    
    print("   ✅ Analyse descriptive avancée terminée")

def visualiser_categorie_indicators(final_data: pd.DataFrame, category: str, indicators: list, export_path: str):
    """Visualisation des indicateurs par catégorie"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Analyse des Indicateurs - Catégorie {category}', fontsize=16, fontweight='bold')
    
    # Graphique 1: Performance moyenne par scénario
    scenario_means = final_data.groupby('scenario_type')[indicators].mean()
    if not scenario_means.empty:
        scenario_means.plot(kind='bar', ax=axes[0,0])
        axes[0,0].set_title(f'Indicateurs {category} par Scénario')
        axes[0,0].tick_params(axis='x', rotation=45)
        axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0,0].grid(True, alpha=0.3)
    
    # Graphique 2: Comparaison par zone
    zone_means = final_data.groupby('zone_type')[indicators].mean()
    if not zone_means.empty:
        zone_means.plot(kind='bar', ax=axes[0,1])
        axes[0,1].set_title(f'Indicateurs {category} par Zone')
        axes[0,1].tick_params(axis='x', rotation=0)
        axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0,1].grid(True, alpha=0.3)
    
    # Graphique 3: Corrélation avec le CAET
    correlation_data = final_data[['performance_globale'] + indicators].corr()
    caet_correlations = correlation_data['performance_globale'].drop('performance_globale').sort_values(ascending=False)
    
    axes[1,0].barh(range(len(caet_correlations)), caet_correlations.values)
    axes[1,0].set_yticks(range(len(caet_correlations)))
    axes[1,0].set_yticklabels([performance_indicators.get(ind, ind) for ind in caet_correlations.index])
    axes[1,0].set_xlabel('Corrélation avec CAET')
    axes[1,0].set_title('Corrélation des Indicateurs avec le Score CAET')
    axes[1,0].grid(True, alpha=0.3)
    
    # Graphique 4: Distribution des indicateurs principaux
    if len(indicators) >= 2:
        main_indicators = indicators[:2]  # Prendre les 2 premiers indicateurs
        for indicator in main_indicators:
            axes[1,1].hist(final_data[indicator], alpha=0.7, label=performance_indicators.get(indicator, indicator), bins=20)
        axes[1,1].set_xlabel('Valeur')
        axes[1,1].set_ylabel('Fréquence')
        axes[1,1].set_title('Distribution des Indicateurs Principaux')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(export_path, f'analyse_categorie_{category}.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()

def analyse_correlation_complete(model_data: pd.DataFrame, available_indicators: list, export_path: str):
    """Analyse complète des corrélations entre tous les indicateurs"""
    
    final_data = model_data[model_data['year'] == 2045]
    
    if final_data.empty or len(available_indicators) < 2:
        return
    
    print("   🔗 Analyse des corrélations...")
    
    # Sélectionner les colonnes pour l'analyse de corrélation
    correlation_columns = ['performance_globale', 'adopts_agroecology'] + available_indicators
    correlation_data = final_data[correlation_columns].corr()
    
    # Créer une heatmap de corrélation
    plt.figure(figsize=(14, 12))
    mask = np.triu(np.ones_like(correlation_data, dtype=bool))  # Masquer la partie supérieure
    sns.heatmap(correlation_data, mask=mask, annot=True, cmap='RdBu_r', center=0,
                square=True, fmt='.2f', cbar_kws={"shrink": .8})
    
    # Renommer les labels pour plus de clarté
    labels = ['CAET', 'Adoption'] + [performance_indicators.get(ind, ind) for ind in available_indicators]
    plt.xticks(range(len(labels)), labels, rotation=45, ha='right')
    plt.yticks(range(len(labels)), labels, rotation=0)
    
    plt.title('Matrice de Corrélation Complète - Tous les Indicateurs (2045)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(export_path, 'matrice_correlation_complete.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    # Analyser les corrélations les plus fortes avec le CAET
    caet_correlations = correlation_data['performance_globale'].drop('performance_globale').sort_values(ascending=False)
    
    # Sauvegarder les corrélations
    correlation_df = pd.DataFrame({
        'Indicateur': caet_correlations.index,
        'Correlation_CAET': caet_correlations.values,
        'Description': [performance_indicators.get(ind, ind) for ind in caet_correlations.index]
    })
    correlation_df.to_excel(os.path.join(export_path, 'correlations_avec_caet.xlsx'), index=False)
    
    print(f"   ✅ Analyse de corrélation terminée - {len(available_indicators)} indicateurs analysés")

def analyse_par_categorie(model_data: pd.DataFrame, available_categories: dict, export_path: str):
    """Analyse détaillée par catégorie d'indicateurs"""
    
    final_data = model_data[model_data['year'] == 2045]
    
    if final_data.empty:
        return
    
    print("   📁 Analyse par catégorie d'indicateurs...")
    
    for category, indicators in available_categories.items():
        if len(indicators) > 0:
            analyser_performance_categorie(final_data, category, indicators, export_path)

def analyser_performance_categorie(final_data: pd.DataFrame, category: str, indicators: list, export_path: str):
    """Analyse de performance pour une catégorie spécifique"""
    
    # Calculer un score composite pour la catégorie
    category_scores = []
    
    for zone in final_data['zone_type'].unique():
        zone_data = final_data[final_data['zone_type'] == zone]
        
        for scenario in zone_data['scenario_type'].unique():
            scenario_data = zone_data[zone_data['scenario_type'] == scenario]
            
            if len(scenario_data) > 0:
                # Calculer la moyenne normalisée des indicateurs de la catégorie
                category_values = []
                for indicator in indicators:
                    if indicator in scenario_data.columns:
                        # Normaliser entre 0 et 1 (en supposant que les valeurs sont déjà dans une échelle comparable)
                        values = scenario_data[indicator]
                        if values.max() > values.min():  # Éviter la division par zéro
                            normalized = (values - values.min()) / (values.max() - values.min())
                        else:
                            normalized = values * 0  # Toutes les valeurs sont identiques
                        category_values.append(normalized.mean())
                
                if category_values:
                    category_score = np.mean(category_values) * 100
                    
                    category_scores.append({
                        'Zone': zone,
                        'Scénario': scenario,
                        'Catégorie': category,
                        'Score_Catégorie': category_score,
                        'Nb_Indicateurs': len(category_values)
                    })
    
    if category_scores:
        # Sauvegarder les scores par catégorie
        scores_df = pd.DataFrame(category_scores)
        scores_df.to_excel(os.path.join(export_path, f'scores_categorie_{category}.xlsx'), index=False)
        
        # Visualiser les scores par catégorie
        plt.figure(figsize=(12, 6))
        
        for zone in scores_df['Zone'].unique():
            zone_scores = scores_df[scores_df['Zone'] == zone]
            plt.bar(zone_scores['Scénario'], zone_scores['Score_Catégorie'], 
                   alpha=0.7, label=zone)
        
        plt.xlabel('Scénario')
        plt.ylabel('Score de la Catégorie (%)')
        plt.title(f'Performance de la Catégorie: {category}')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(export_path, f'performance_categorie_{category}.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()

def generer_rapport_complet(model_data: pd.DataFrame, available_indicators: list, available_categories: dict, export_path: str):
    """Génération d'un rapport complet d'analyse"""
    
    rapport_file = os.path.join(export_path, "rapport_analyse_complete.md")
    
    with open(rapport_file, 'w', encoding='utf-8') as f:
        f.write("# Rapport Complet d'Analyse des Résultats de Simulation\n\n")
        f.write(f"*Généré le {datetime.now().strftime('%d/%m/%Y à %H:%M')}*\n\n")
        
        # Résumé exécutif
        f.write("## Résumé Exécutif\n\n")
        
        final_data = model_data[model_data['year'] == 2045]
        if not final_data.empty:
            best_scenario_opib = final_data[final_data['zone_type'] == 'OPIB'].groupby('scenario_type')['performance_globale'].mean().idxmax()
            best_scenario_ohvn = final_data[final_data['zone_type'] == 'OHVN'].groupby('scenario_type')['performance_globale'].mean().idxmax()
            
            f.write(f"- **Scénario recommandé pour OPIB**: {best_scenario_opib}\n")
            f.write(f"- **Scénario recommandé pour OHVN**: {best_scenario_ohvn}\n")
            f.write(f"- **Indicateurs analysés**: {len(available_indicators)}\n")
            f.write(f"- **Catégories couvertes**: {len(available_categories)}\n\n")
        
        # Détail des indicateurs disponibles
        f.write("## Indicateurs de Performance Disponibles\n\n")
        
        for category, indicators in available_categories.items():
            f.write(f"### {category}\n\n")
            for indicator in indicators:
                description = performance_indicators.get(indicator, indicator)
                f.write(f"- **{indicator}**: {description}\n")
            f.write("\n")
        
        # Analyse des corrélations
        f.write("## Analyse des Corrélations\n\n")
        f.write("Les corrélations entre les indicateurs et le score CAET global sont disponibles dans le fichier 'correlations_avec_caet.xlsx'.\n\n")
        
        # Recommandations stratégiques
        f.write("## Recommandations Stratégiques par Catégorie\n\n")
        
        recommendations = {
            'Économique': "Focus sur la rentabilité et l'accès aux marchés",
            'Environnemental': "Priorité à la santé des sols et la biodiversité", 
            'Social': "Renforcement de la cohésion sociale et de la participation",
            'Territorial': "Développement des infrastructures et connectivité",
            'Alimentation': "Amélioration de la diversité alimentaire et souveraineté"
        }
        
        for category in available_categories.keys():
            if category in recommendations:
                f.write(f"### {category}\n\n")
                f.write(f"{recommendations[category]}\n\n")
        
        f.write("## Méthodologie d'Analyse\n\n")
        f.write("- **Période analysée**: 2045 (résultats finaux)\n")
        f.write("- **Indicateurs**: Analyse descriptive et corrélations\n") 
        f.write("- **Visualisations**: Graphiques par catégorie et scénario\n")
        f.write("- **Scores composites**: Calcul par catégorie d'indicateurs\n")
    
    print("   ✅ Rapport complet généré")

# === LANCEMENT DE L'ANALYSE COMPLÈTE ===
if __name__ == "__main__":
    """
    Lance l'analyse complète avec tous les indicateurs de performance
    """
    print("🌍 DÉMARRAGE DE L'ANALYSE COMPLÈTE AVEC TOUS LES INDICATEURS")
    
    # Chemin vers votre fichier de résultats
    results_file = "simulation_complete_results/simulation_complete_2025_2045.xlsx"
    
    # Lancer l'analyse complète
    results = analyse_globale_resultats_complete(
        excel_file=results_file,
        export_path="analyse_complete_avec_tous_indicateurs"
    )
    
    if results:
        print(f"\n✅ ANALYSE COMPLÈTE RÉUSSIE!")
        print(f"📊 Indicateurs analysés: {len(results['available_indicators'])}")
        print(f"📁 Catégories: {len(results['available_categories'])}")
        print(f"💾 Dossier résultats: {results['export_path']}")
    else:
        print("\n❌ L'analyse complète a échoué")

🌍 DÉMARRAGE DE L'ANALYSE COMPLÈTE AVEC TOUS LES INDICATEURS
🌍 ANALYSE GLOBALE COMPLÈTE DES RÉSULTATS
📂 CHARGEMENT DES DONNÉES...
✅ Données chargées - Modèles: 35910 lignes, Agents: 35910 lignes

🔍 IDENTIFICATION DES INDICATEURS DISPONIBLES...
   ✅ Indicateurs essentiels: 5
   ✅ Indicateurs de performance disponibles: 10
   📊 Liste: dietary_diversity, soil_health, landtenure, pest_score, Agricultural_diversity_tape...

📈 ANALYSE PAR CATÉGORIES D'INDICATEURS...
   📁 Économique: 2 indicateurs
   📁 Environnemental: 1 indicateurs
   📁 Social: 2 indicateurs
   📁 Alimentation: 1 indicateurs
   📊 Analyse descriptive avancée...
   ✅ Analyse descriptive avancée terminée
   🔗 Analyse des corrélations...
   ✅ Analyse de corrélation terminée - 10 indicateurs analysés
   📁 Analyse par catégorie d'indicateurs...
   ✅ Rapport complet généré

✅ ANALYSE GLOBALE COMPLÈTE TERMINÉE!
📁 Dossier résultats: analyse_complete_avec_tous_indicateurs

✅ ANALYSE COMPLÈTE RÉUSSIE!
📊 Indicateurs analysés: 10
📁 Catégor

In [5]:
# === IMPORTS GLOBAUX ===
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# === ANALYSE GLOBALE COMPLÈTE PAR ZONE ET SCÉNARIO - VERSION CORRIGÉE ===

def analyse_globale_complete(excel_file: str, export_path: str = "analyse_complete_simulation"):
    """
    Analyse globale complète avec analyse spécifique par zone et scénario
    Version finale corrigée avec imports globaux
    """
    
    if not os.path.exists(export_path):
        os.makedirs(export_path)
    
    print("🌍 ANALYSE GLOBALE COMPLÈTE PAR ZONE ET SCÉNARIO")
    print("=" * 60)
    
    # 1. CHARGEMENT ET VÉRIFICATION DES DONNÉES
    print("📂 CHARGEMENT ET VÉRIFICATION DES DONNÉES...")
    
    try:
        # Chargement des données principales
        model_data = pd.read_excel(excel_file, sheet_name='Resultats_Modeles')
        agent_data = pd.read_excel(excel_file, sheet_name='Donnees_Agents_Details')
        
        print(f"✅ Données chargées - Modèles: {len(model_data)} lignes, Agents: {len(agent_data)} lignes")
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return None
    
    # 2. ANALYSE SPÉCIFIQUE PAR ZONE ET SCÉNARIO
    print("\n🎯 ANALYSE SPÉCIFIQUE PAR ZONE ET SCÉNARIO...")
    
    # Créer des sous-dossiers par zone
    zones = model_data['zone_type'].unique()
    for zone in zones:
        zone_path = os.path.join(export_path, f"zone_{zone}")
        if not os.path.exists(zone_path):
            os.makedirs(zone_path)
    
    # 3. ANALYSES PAR ZONE
    print("\n🌍 ANALYSE COMPARATIVE PAR ZONE...")
    analyse_comparative_zones(model_data, export_path)
    
    # 4. ANALYSE DES DÉTERMINANTS PAR ZONE ET SCÉNARIO
    print("\n🔍 ANALYSE DES DÉTERMINANTS PAR ZONE ET SCÉNARIO...")
    analyse_determinants_par_zone_scenario(model_data, agent_data, export_path)
    
    # 5. ANALYSE DES PERFORMANCES PAR SCÉNARIO
    print("\n📊 ANALYSE DES PERFORMANCES PAR SCÉNARIO...")
    analyse_performances_par_scenario(model_data, export_path)
    
    # 6. ANALYSE DES TRAJECTOIRES PAR ZONE
    print("\n🔄 ANALYSE DES TRAJECTOIRES PAR ZONE...")
    analyse_trajectoires_par_zone(model_data, export_path)
    
    # 7. RAPPORT COMPARATIF ZONE/SCÉNARIO
    print("\n📝 GÉNÉRATION DU RAPPORT COMPARATIF...")
    generer_rapport_comparatif_zones_scenarios(model_data, agent_data, export_path)
    
    print(f"\n✅ ANALYSE PAR ZONE ET SCÉNARIO TERMINÉE AVEC SUCCÈS!")
    print(f"📁 Dossier résultats: {export_path}")
    
    return {
        'model_data': model_data,
        'agent_data': agent_data,
        'zones': zones,
        'scenarios': model_data['scenario_type'].unique(),
        'export_path': export_path
    }

def analyse_comparative_zones(model_data: pd.DataFrame, export_path: str):
    """Analyse comparative détaillée entre les zones"""
    
    final_data = model_data[model_data['year'] == 2045]
    
    if final_data.empty:
        print("   ⚠️ Aucune donnée pour 2045")
        return
    
    zones = final_data['zone_type'].unique()
    
    print(f"   🔍 Comparaison de {len(zones)} zones")
    
    # 1. COMPARAISON GLOBALE DES ZONES
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('COMPARAISON DES PERFORMANCES PAR ZONE (2045)', fontsize=16, fontweight='bold')
    
    # Graphique 1: Distribution du CAET par zone
    zone_data_list = []
    zone_labels = []
    for zone in zones:
        zone_data = final_data[final_data['zone_type'] == zone]
        zone_data_list.append(zone_data['performance_globale'] * 100)
        zone_labels.append(zone)
    
    axes[0,0].boxplot(zone_data_list, labels=zone_labels)
    axes[0,0].set_ylabel('Score CAET (%)')
    axes[0,0].set_title('Distribution du CAET par Zone')
    axes[0,0].grid(True, alpha=0.3)
    
    # Graphique 2: Taux d'adoption par zone
    adoption_means = final_data.groupby('zone_type')['adopts_agroecology'].mean() * 100
    bars = axes[0,1].bar(adoption_means.index, adoption_means.values, 
                        color=['skyblue', 'lightcoral'], alpha=0.7)
    axes[0,1].set_ylabel('Taux d\'Adoption (%)')
    axes[0,1].set_title('Taux d\'Adoption Agroécologique par Zone')
    axes[0,1].grid(True, alpha=0.3)
    
    # Ajout des valeurs sur les barres
    for bar in bars:
        height = bar.get_height()
        axes[0,1].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                      f'{height:.1f}%', ha='center', va='bottom')
    
    # Graphique 3: Performance moyenne par zone
    caet_means = final_data.groupby('zone_type')['performance_globale'].mean() * 100
    bars = axes[0,2].bar(caet_means.index, caet_means.values,
                        color=['lightgreen', 'gold'], alpha=0.7)
    axes[0,2].set_ylabel('Score CAET Moyen (%)')
    axes[0,2].set_title('Performance CAET Moyenne par Zone')
    axes[0,2].grid(True, alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        axes[0,2].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                      f'{height:.1f}%', ha='center', va='bottom')
    
    # Graphique 4: Comparaison des scénarios entre zones
    scenario_comparison = final_data.groupby(['zone_type', 'scenario_type'])['performance_globale'].mean().unstack()
    if not scenario_comparison.empty:
        scenario_comparison.T.plot(kind='bar', ax=axes[1,0])
        axes[1,0].set_ylabel('Score CAET Moyen (%)')
        axes[1,0].set_title('Performance des Scénarios par Zone')
        axes[1,0].tick_params(axis='x', rotation=45)
        axes[1,0].legend(title='Zone')
        axes[1,0].grid(True, alpha=0.3)
    
    # Graphique 5: Évolution temporelle par zone
    evolution_data = model_data.groupby(['year', 'zone_type'])['performance_globale'].mean().unstack()
    if not evolution_data.empty:
        evolution_data.plot(ax=axes[1,1], linewidth=2.5, marker='o')
        axes[1,1].set_xlabel('Année')
        axes[1,1].set_ylabel('Score CAET Moyen')
        axes[1,1].set_title('Évolution du CAET par Zone (2025-2045)')
        axes[1,1].legend(title='Zone')
        axes[1,1].grid(True, alpha=0.3)
    
    # Graphique 6: Heatmap de performance zone/scénario
    try:
        heatmap_data = final_data.groupby(['zone_type', 'scenario_type'])['performance_globale'].mean().unstack()
        if not heatmap_data.empty:
            sns.heatmap(heatmap_data * 100, annot=True, fmt='.1f', cmap='YlOrRd', 
                       ax=axes[1,2], cbar_kws={'label': 'Score CAET (%)'})
            axes[1,2].set_title('Performance Zone/Scénario')
    except Exception as e:
        axes[1,2].text(0.5, 0.5, f'Erreur heatmap:\n{str(e)[:30]}', 
                      ha='center', va='center', transform=axes[1,2].transAxes)
    
    plt.tight_layout()
    plt.savefig(os.path.join(export_path, 'comparaison_zones_global.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. ANALYSE DÉTAILLÉE PAR ZONE
    for zone in zones:
        print(f"   📊 Analyse détaillée de la zone {zone}...")
        analyser_zone_specifique(model_data, zone, export_path)
    
    print("   ✅ Analyse comparative des zones terminée")

def analyser_zone_specifique(model_data: pd.DataFrame, zone: str, export_path: str):
    """Analyse détaillée pour une zone spécifique"""
    
    zone_data = model_data[model_data['zone_type'] == zone]
    final_data = zone_data[zone_data['year'] == 2045]
    
    if final_data.empty:
        return
    
    zone_path = os.path.join(export_path, f"zone_{zone}")
    
    # Créer les visualisations spécifiques à la zone
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'ANALYSE DÉTAILLÉE - ZONE {zone}', fontsize=16, fontweight='bold')
    
    # Graphique 1: Classement des scénarios
    scenario_means = final_data.groupby('scenario_type')['performance_globale'].mean().sort_values(ascending=False)
    bars = axes[0,0].bar(range(len(scenario_means)), scenario_means.values * 100, 
                        color=plt.cm.viridis(np.linspace(0, 1, len(scenario_means))))
    axes[0,0].set_xlabel('Scénarios')
    axes[0,0].set_ylabel('Score CAET (%)')
    axes[0,0].set_title(f'Classement des Scénarios - {zone}')
    axes[0,0].set_xticks(range(len(scenario_means)))
    axes[0,0].set_xticklabels(scenario_means.index, rotation=45, ha='right')
    axes[0,0].grid(True, alpha=0.3)
    
    # Ajout des valeurs
    for i, bar in enumerate(bars):
        height = bar.get_height()
        axes[0,0].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                      f'{height:.1f}%', ha='center', va='bottom', fontsize=9)
    
    # Graphique 2: Évolution des scénarios
    evolution_scenarios = zone_data.groupby(['year', 'scenario_type'])['performance_globale'].mean().unstack()
    if not evolution_scenarios.empty:
        evolution_scenarios.plot(ax=axes[0,1], linewidth=2, marker='o', markersize=4)
        axes[0,1].set_xlabel('Année')
        axes[0,1].set_ylabel('Score CAET')
        axes[0,1].set_title(f'Évolution des Scénarios - {zone}')
        axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0,1].grid(True, alpha=0.3)
    
    # Graphique 3: Taux d'adoption par scénario
    adoption_scenarios = final_data.groupby('scenario_type')['adopts_agroecology'].mean() * 100
    bars = axes[1,0].bar(range(len(adoption_scenarios)), adoption_scenarios.values,
                        color=plt.cm.Set3(np.linspace(0, 1, len(adoption_scenarios))))
    axes[1,0].set_xlabel('Scénarios')
    axes[1,0].set_ylabel('Taux d\'Adoption (%)')
    axes[1,0].set_title(f'Adoption Agroécologique par Scénario - {zone}')
    axes[1,0].set_xticks(range(len(adoption_scenarios)))
    axes[1,0].set_xticklabels(adoption_scenarios.index, rotation=45, ha='right')
    axes[1,0].grid(True, alpha=0.3)
    
    for i, bar in enumerate(bars):
        height = bar.get_height()
        axes[1,0].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                      f'{height:.1f}%', ha='center', va='bottom', fontsize=9)
    
    # Graphique 4: Gains 2025-2045 par scénario
    gains_data = []
    initial_data = zone_data[zone_data['year'] == 2025]
    
    for scenario in final_data['scenario_type'].unique():
        scenario_final = final_data[final_data['scenario_type'] == scenario]
        scenario_initial = initial_data[initial_data['scenario_type'] == scenario]
        
        if not scenario_final.empty and not scenario_initial.empty:
            gain = (scenario_final['performance_globale'].mean() - 
                   scenario_initial['performance_globale'].mean()) * 100
            gains_data.append({'Scénario': scenario, 'Gain': gain})
    
    if gains_data:
        gains_df = pd.DataFrame(gains_data).sort_values('Gain', ascending=False)
        bars = axes[1,1].bar(range(len(gains_df)), gains_df['Gain'],
                           color=plt.cm.coolwarm(np.linspace(0, 1, len(gains_df))))
        axes[1,1].set_xlabel('Scénarios')
        axes[1,1].set_ylabel('Gain CAET 2025-2045 (%)')
        axes[1,1].set_title(f'Gains de Performance par Scénario - {zone}')
        axes[1,1].set_xticks(range(len(gains_df)))
        axes[1,1].set_xticklabels(gains_df['Scénario'], rotation=45, ha='right')
        axes[1,1].grid(True, alpha=0.3)
        
        for i, bar in enumerate(bars):
            height = bar.get_height()
            axes[1,1].text(bar.get_x() + bar.get_width()/2., height + (0.1 if height >= 0 else -0.5),
                          f'{height:+.1f}%', ha='center', va='bottom' if height >= 0 else 'top', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(zone_path, f'analyse_detaille_{zone}.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    # Sauvegarde des données détaillées
    stats_zone = final_data.groupby('scenario_type').agg({
        'performance_globale': ['mean', 'std', 'min', 'max'],
        'adopts_agroecology': 'mean'
    }).round(3)
    
    stats_zone.to_excel(os.path.join(zone_path, f'statistiques_{zone}.xlsx'))

def analyse_determinants_par_zone_scenario(model_data: pd.DataFrame, agent_data: pd.DataFrame, export_path: str):
    """Analyse des déterminants du CAET spécifique par zone et scénario"""
    
    if agent_data.empty:
        print("   ⚠️ Données agents manquantes pour l'analyse des déterminants")
        return
    
    final_agents = agent_data[agent_data['year'] == 2045]
    
    if final_agents.empty:
        print("   ⚠️ Aucune donnée d'agents pour 2045")
        return
    
    zones = final_agents['zone_type'].unique()
    
    for zone in zones:
        print(f"   🔍 Analyse des déterminants pour {zone}...")
        zone_agents = final_agents[final_agents['zone_type'] == zone]
        
        # Analyser par scénario dans la zone
        scenarios = zone_agents['scenario_type'].unique()
        
        for scenario in scenarios:
            scenario_agents = zone_agents[zone_agents['scenario_type'] == scenario]
            
            if len(scenario_agents) > 10:  # Minimum d'observations
                try:
                    analyser_determinants_scenario(scenario_agents, zone, scenario, export_path)
                except Exception as e:
                    print(f"      ⚠️ Erreur pour {zone}-{scenario}: {e}")
            else:
                print(f"      ⚠️ Données insuffisantes pour {zone}-{scenario}")

def analyser_determinants_scenario(agents_data: pd.DataFrame, zone: str, scenario: str, export_path: str):
    """Analyse des déterminants pour un scénario spécifique"""
    
    try:
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import r2_score
        
        # Variables potentielles
        potential_predictors = [
            'farm_area', 'hh_men', 'hh_women', 'innovation_capacity', 
            'social_network', 'community_engagement', 'systemic_thinking'
        ]
        
        # Vérifier les variables disponibles
        available_predictors = [var for var in potential_predictors if var in agents_data.columns]
        
        if len(available_predictors) < 2:
            return
        
        # Préparation des données
        X = agents_data[available_predictors].copy()
        
        # Variable cible
        if 'caet_tot' in agents_data.columns:
            y = agents_data['caet_tot'].copy()
        elif 'performance_globale' in agents_data.columns:
            y = agents_data['performance_globale'].copy() * 100
        else:
            return
        
        # Nettoyage
        X = X.replace([np.inf, -np.inf], np.nan)
        y = y.replace([np.inf, -np.inf], np.nan)
        
        valid_mask = X.notna().all(axis=1) & y.notna()
        X_clean = X[valid_mask]
        y_clean = y[valid_mask]
        
        if len(X_clean) < 5:
            return
        
        # Modèle simple
        X_train, X_test, y_train, y_test = train_test_split(
            X_clean, y_clean, test_size=0.3, random_state=42
        )
        
        rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
        rf_model.fit(X_train, y_train)
        
        y_pred = rf_model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        
        # Importance des variables
        importance_df = pd.DataFrame({
            'Variable': available_predictors,
            'Importance': rf_model.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        # Visualisation
        plt.figure(figsize=(10, 6))
        bars = plt.barh(range(len(importance_df)), importance_df['Importance'],
                       color=plt.cm.viridis(np.linspace(0, 1, len(importance_df))))
        plt.yticks(range(len(importance_df)), importance_df['Variable'])
        plt.xlabel('Importance')
        plt.title(f'Déterminants du CAET - {zone} - {scenario}\n(R² = {r2:.3f})')
        plt.grid(True, alpha=0.3)
        
        for i, bar in enumerate(bars):
            width = bar.get_width()
            plt.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                    f'{width:.3f}', ha='left', va='center', fontsize=9)
        
        plt.tight_layout()
        
        # Sauvegarde dans le dossier de la zone
        zone_path = os.path.join(export_path, f"zone_{zone}")
        plt.savefig(os.path.join(zone_path, f'determinants_{zone}_{scenario}.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # Sauvegarde des données
        importance_df.to_excel(os.path.join(zone_path, f'importance_variables_{zone}_{scenario}.xlsx'), 
                              index=False)
        
    except Exception as e:
        print(f"      ❌ Erreur analyse déterminants {zone}-{scenario}: {e}")

def analyse_performances_par_scenario(model_data: pd.DataFrame, export_path: str):
    """Analyse comparative des performances par scénario"""
    
    final_data = model_data[model_data['year'] == 2045]
    
    if final_data.empty:
        return
    
    scenarios = final_data['scenario_type'].unique()
    
    print(f"   📈 Analyse de {len(scenarios)} scénarios")
    
    # Classement global des scénarios
    scenario_performance = final_data.groupby('scenario_type').agg({
        'performance_globale': ['mean', 'std', 'count'],
        'adopts_agroecology': 'mean'
    }).round(3)
    
    scenario_performance.columns = ['CAET_moyen', 'CAET_std', 'Nb_agents', 'Taux_adoption']
    scenario_performance['CAET_moyen'] = scenario_performance['CAET_moyen'] * 100
    scenario_performance['Taux_adoption'] = scenario_performance['Taux_adoption'] * 100
    
    # Classement par performance
    scenario_ranking = scenario_performance.sort_values('CAET_moyen', ascending=False)
    
    # Visualisation du classement
    plt.figure(figsize=(12, 8))
    bars = plt.barh(range(len(scenario_ranking)), scenario_ranking['CAET_moyen'],
                   color=plt.cm.RdYlGn(np.linspace(0, 1, len(scenario_ranking))))
    
    plt.yticks(range(len(scenario_ranking)), scenario_ranking.index)
    plt.xlabel('Score CAET Moyen (%)')
    plt.title('CLASSEMENT DES SCÉNARIOS PAR PERFORMANCE CAET (2045)')
    plt.grid(True, alpha=0.3)
    
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + 0.5, bar.get_y() + bar.get_height()/2, 
                f'{width:.1f}%', ha='left', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(os.path.join(export_path, 'classement_scenarios_global.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    # Sauvegarde du classement
    scenario_ranking.to_excel(os.path.join(export_path, 'classement_scenarios.xlsx'))

def analyse_trajectoires_par_zone(model_data: pd.DataFrame, export_path: str):
    """Analyse des trajectoires d'évolution par zone"""
    
    zones = model_data['zone_type'].unique()
    
    fig, axes = plt.subplots(1, len(zones), figsize=(5*len(zones), 6))
    if len(zones) == 1:
        axes = [axes]
    
    for i, zone in enumerate(zones):
        zone_data = model_data[model_data['zone_type'] == zone]
        
        # Évolution moyenne par scénario
        evolution_scenarios = zone_data.groupby(['year', 'scenario_type'])['performance_globale'].mean().unstack()
        
        if not evolution_scenarios.empty:
            evolution_scenarios.plot(ax=axes[i], linewidth=2.5, marker='o', markersize=4)
            axes[i].set_xlabel('Année')
            axes[i].set_ylabel('Score CAET')
            axes[i].set_title(f'Trajectoires - {zone}')
            axes[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(export_path, 'trajectoires_zones.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()

def generer_rapport_comparatif_zones_scenarios(model_data: pd.DataFrame, agent_data: pd.DataFrame, export_path: str):
    """Génération d'un rapport comparatif zones/scénarios"""
    
    rapport_file = os.path.join(export_path, "rapport_comparatif_zones_scenarios.md")
    
    with open(rapport_file, 'w', encoding='utf-8') as f:
        f.write("# Rapport Comparatif Zones et Scénarios\n\n")
        f.write(f"*Généré le {datetime.now().strftime('%d/%m/%Y à %H:%M')}*\n\n")
        
        final_data = model_data[model_data['year'] == 2045]
        
        if not final_data.empty:
            # Analyse par zone
            f.write("## Analyse par Zone\n\n")
            
            for zone in final_data['zone_type'].unique():
                zone_data = final_data[final_data['zone_type'] == zone]
                best_scenario = zone_data.groupby('scenario_type')['performance_globale'].mean().idxmax()
                best_score = zone_data.groupby('scenario_type')['performance_globale'].mean().max() * 100
                
                f.write(f"### {zone}\n\n")
                f.write(f"- **Scénario optimal**: {best_scenario} ({best_score:.1f}% CAET)\n")
                f.write(f"- **Performance moyenne**: {zone_data['performance_globale'].mean()*100:.1f}%\n")
                f.write(f"- **Taux d'adoption**: {zone_data['adopts_agroecology'].mean()*100:.1f}%\n")
                f.write(f"- **Nombre d'agents**: {len(zone_data)}\n\n")
            
            # Recommandations
            f.write("## Recommandations Stratégiques\n\n")
            
            for zone in final_data['zone_type'].unique():
                zone_data = final_data[final_data['zone_type'] == zone]
                best_scenario = zone_data.groupby('scenario_type')['performance_globale'].mean().idxmax()
                
                f.write(f"### Pour la zone {zone}\n\n")
                f.write(f"**Scénario recommandé**: {best_scenario}\n\n")
                
                if "baguineda" in best_scenario.lower():
                    f.write("- Prioriser les politiques de développement agricole durable\n")
                    f.write("- Renforcer les infrastructures agroécologiques\n")
                    f.write("- Favoriser l'accès aux marchés locaux\n\n")
                elif "ohvn" in best_scenario.lower():
                    f.write("- Développer l'agro-industrie responsable\n")
                    f.write("- Investir dans l'innovation technologique\n")
                    f.write("- Renforcer les circuits de commercialisation\n\n")
    
    print("   ✅ Rapport comparatif généré")

# === LANCEMENT DE L'ANALYSE PAR ZONE ET SCÉNARIO ===
if __name__ == "__main__":
    """
    Lance l'analyse spécifique par zone et scénario
    """
    print("🌍 DÉMARRAGE DE L'ANALYSE PAR ZONE ET SCÉNARIO")
    
    # Chemin vers votre fichier de résultats
    results_file = "simulation_complete_results/simulation_complete_2025_2045.xlsx"
    
    # Lancer l'analyse
    results = analyse_globale_complete(
        excel_file=results_file,
        export_path="analyse_par_zone_scenario"
    )
    
    if results:
        print(f"\n✅ ANALYSE PAR ZONE ET SCÉNARIO RÉUSSIE!")
        print(f"🌍 Zones analysées: {len(results['zones'])}")
        print(f"🎯 Scénarios analysés: {len(results['scenarios'])}")
        print(f"📁 Dossier résultats: {results['export_path']}")
    else:
        print("\n❌ L'analyse a échoué")

🌍 DÉMARRAGE DE L'ANALYSE PAR ZONE ET SCÉNARIO
🌍 ANALYSE GLOBALE COMPLÈTE PAR ZONE ET SCÉNARIO
📂 CHARGEMENT ET VÉRIFICATION DES DONNÉES...
✅ Données chargées - Modèles: 35910 lignes, Agents: 35910 lignes

🎯 ANALYSE SPÉCIFIQUE PAR ZONE ET SCÉNARIO...

🌍 ANALYSE COMPARATIVE PAR ZONE...
   🔍 Comparaison de 2 zones
   📊 Analyse détaillée de la zone OPIB...
   📊 Analyse détaillée de la zone OHVN...
   ✅ Analyse comparative des zones terminée

🔍 ANALYSE DES DÉTERMINANTS PAR ZONE ET SCÉNARIO...
   🔍 Analyse des déterminants pour OPIB...
   🔍 Analyse des déterminants pour OHVN...

📊 ANALYSE DES PERFORMANCES PAR SCÉNARIO...
   📈 Analyse de 11 scénarios

🔄 ANALYSE DES TRAJECTOIRES PAR ZONE...

📝 GÉNÉRATION DU RAPPORT COMPARATIF...
   ✅ Rapport comparatif généré

✅ ANALYSE PAR ZONE ET SCÉNARIO TERMINÉE AVEC SUCCÈS!
📁 Dossier résultats: analyse_par_zone_scenario

✅ ANALYSE PAR ZONE ET SCÉNARIO RÉUSSIE!
🌍 Zones analysées: 2
🎯 Scénarios analysés: 11
📁 Dossier résultats: analyse_par_zone_scenario
